Celda 1 — Entorno limpio + acceso a tus Google Sheets

Qué hace: instala lo justo, te autentica en Colab, y verifica que podemos abrir tus dos hojas por ID, mostrando sus pestañas.


In [1]:

!pip -q install gspread google-auth==2.38.0 folium ipywidgets

# 1) Autenticación con tu cuenta de Google
from google.colab import auth
auth.authenticate_user()
print("OK: autenticación realizada.")

# 2) Conectar a Google Sheets
import gspread, google.auth

SCOPES = ["https://www.googleapis.com/auth/spreadsheets",
          "https://www.googleapis.com/auth/drive"]
creds, _ = google.auth.default(scopes=SCOPES)
gc = gspread.authorize(creds)

# 3) IDs y pestañas preferidas (tuyas, ya fijas)
GAZ_ID  = "1eLHEJ7N1_uIfPnwBTIplzeWPz_YfBOljJGaIYp56jlw"   # Hoja maestra / gazetteer
MINI_ID = "12aHBLMfcCvgEokCwt28tdomimTfGPCkKUYKhDYdy7dk"   # Circuitos de minibuses
GAZ_TAB_PREF  = "Santa Fe"
MINI_TAB_PREF = "Salida_Paradas_Geo"

# 4) Probar apertura y listar pestañas
def try_open(sheet_id, etiqueta):
    try:
        sh = gc.open_by_key(sheet_id.strip())
        tabs = [ws.title for ws in sh.worksheets()]
        print(f"✅ {etiqueta}: '{sh.title}' — pestañas: {tabs}")
        return sh
    except gspread.SpreadsheetNotFound:
        print(f"❌ {etiqueta}: NO se pudo abrir (404/permiso). "
              "Verificá el ID y que esta cuenta tenga acceso como Editor.")
    except Exception as e:
        print(f"❌ {etiqueta}: error inesperado → {e}")

sh_gaz = try_open(GAZ_ID,  "Hoja MAESTRA (gazetteer)")
sh_min = try_open(MINI_ID, "Hoja CIRCUITOS (minibuses)")

# 5) (Opcional) habilitar widgets para más adelante
try:
    from google.colab import output as colab_output
    colab_output.enable_custom_widget_manager()
    print("Widgets OK.")
except Exception as e:
    print("Widgets: no pude habilitar el manager (seguimos igual).", e)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.9 MB/s eta 0:00:00
OK: autenticación realizada.
✅ Hoja MAESTRA (gazetteer): 'Hoja Maestra de Localidades georref' — pestañas: ['Santa Fe']
❌ Hoja CIRCUITOS (minibuses): error inesperado → APIError: [503]: The service is currently unavailable.
Widgets OK.


In [2]:
# === CELDA 2: leer gazetteer y minibuses + diagnóstico ===
import pandas as pd, numpy as np, re, unicodedata, math
import gspread

# Usa los objetos/constantes creados en la Celda 1:
# gc, GAZ_ID, MINI_ID, GAZ_TAB_PREF, MINI_TAB_PREF

def open_df(file_id: str, preferred_tab: str):
    """Abre la hoja por ID, usa la pestaña pedida o cae a la 1ra.
    Devuelve (titulo_hoja, pestaña_usada, df, lista_de_pestañas)."""
    sh = gc.open_by_key(file_id.strip())
    tabs = [ws.title for ws in sh.worksheets()]
    if preferred_tab in tabs:
        ws = sh.worksheet(preferred_tab)
    else:
        cand = [t for t in tabs if preferred_tab.lower() in t.lower()]
        ws = sh.worksheet(cand[0]) if cand else sh.get_worksheet(0)
    # Traigo todo como registros (evita líos de dependencias)
    rows = ws.get_all_records()
    df = pd.DataFrame(rows).dropna(how="all")
    df.columns = [str(c).strip() for c in df.columns]
    return sh.title, ws.title, df, tabs

# Abrir ambas fuentes
gaz_title, gaz_tab_used, gaz_df_raw, gaz_tabs = open_df(GAZ_ID,  GAZ_TAB_PREF)
mini_title, mini_tab_used, mini_df_raw, mini_tabs = open_df(MINI_ID, MINI_TAB_PREF)

print(f"✅ Gazetteer: '{gaz_title}' — pestaña usada: '{gaz_tab_used}' (disponibles: {gaz_tabs})")
print(f"✅ Minibuses: '{mini_title}' — pestaña usada: '{mini_tab_used}' (disponibles: {mini_tabs})\n")

# Mostrar columnas y tamaños
print("— Resumen de columnas —")
print("GAZETTEER:", list(gaz_df_raw.columns))
print("MINIBUSES:", list(mini_df_raw.columns), "\n")

# Chequeo rápido de columnas esperadas
gaz_must = {"centroide_lat", "centroide_lon"}
gaz_name_cols = [c for c in ["nombre","localidad_censal_nombre","municipio_nombre"] if c in gaz_df_raw.columns]
missing_gaz = sorted(list(gaz_must - set(gaz_df_raw.columns)))
if missing_gaz:
    print("⚠️ Faltan columnas en GAZETTEER:", missing_gaz)
else:
    print("OK GAZETTEER: tiene lat/lon. Campos de nombre disponibles:", gaz_name_cols or "—")

mini_must_any = {"LOCALIDAD DE ORIGEN","LOCALIDAD DE DESTINO"}
missing_mini = sorted(list(mini_must_any - set(mini_df_raw.columns)))
if missing_mini:
    print("⚠️ Faltan columnas clave en MINIBUSES:", missing_mini)
else:
    extra = [c for c in ["PARADAS_GEO","PARADAS INTERMEDIAS","DIAS OPERATIVOS","OBSERVACIONES",
                         "COORDENADAS ORIGEN","COORDENADAS DESTINO",
                         "LAT_ORIGEN","LON_ORIGEN","LAT_DESTINO","LON_DESTINO"]
             if c in mini_df_raw.columns]
    print("OK MINIBUSES: tiene origen/destino. Otras útiles detectadas:", extra or "—")

# Pequeña vista previa para que validemos a ojo
print("\nVista previa GAZETTEER (5 filas):")
display(gaz_df_raw.head(5))
print("\nVista previa MINIBUSES (5 filas):")
display(mini_df_raw.head(5))

# Guardar en variables de trabajo para las próximas celdas
gaz_df = gaz_df_raw.copy()
mini_df = mini_df_raw.copy()
print(f"\nListo: gaz_df={gaz_df.shape}, mini_df={mini_df.shape}")


✅ Gazetteer: 'Hoja Maestra de Localidades georref' — pestaña usada: 'Santa Fe' (disponibles: ['Santa Fe'])
✅ Minibuses: 'Circuitos de los Minibus' — pestaña usada: 'Salida_Paradas_Geo' (disponibles: ['Salida_Paradas_Geo'])

— Resumen de columnas —
GAZETTEER: ['centroide_lat', 'centroide_lon', 'departamento_nombre', 'localidad_censal_nombre', 'municipio_nombre', 'nombre', 'provincia_nombre']
MINIBUSES: ['LOCALIDAD DE ORIGEN', 'COORDENADAS ORIGEN', 'LOCALIDAD DE DESTINO', 'COORDENADAS DESTINO', 'PARADAS INTERMEDIAS', 'DIAS OPERATIVOS', 'OBSERVACIONES', 'PARADAS_GEO', 'PARADAS INTERMEDIAS (CORREGIDAS)', 'LAT_ORIGEN', 'LON_ORIGEN', 'LAT_DESTINO', 'LON_DESTINO'] 

OK GAZETTEER: tiene lat/lon. Campos de nombre disponibles: ['nombre', 'localidad_censal_nombre', 'municipio_nombre']
OK MINIBUSES: tiene origen/destino. Otras útiles detectadas: ['PARADAS_GEO', 'PARADAS INTERMEDIAS', 'DIAS OPERATIVOS', 'OBSERVACIONES', 'COORDENADAS ORIGEN', 'COORDENADAS DESTINO', 'LAT_ORIGEN', 'LON_ORIGEN', 'LAT_D

,centroide_lat,centroide_lon,departamento_nombre,localidad_censal_nombre,municipio_nombre,nombre,provincia_nombre
0,-312.142.150.140.904,-616.129.125.462.879,Castellanos,Presidente Roca,Presidente Roca,Presidente Roca,Santa Fe
1,-312.321.284.987.409,-616.102.774.667.774,Castellanos,Presidente Roca,Presidente Roca,Estación Presidente Roca,Santa Fe
2,-317.210.301.786.451,-608.015.887.552.447,La Capital,Sauce Viejo,Sauce Viejo,Villa Adelina,Santa Fe
3,-317.660.371.876.277,-608.305.595.552.308,La Capital,Sauce Viejo,Sauce Viejo,Sauce Viejo,Santa Fe
4,-315.604.450.839.944,-605.191.788.520.517,La Capital,Arroyo Leyes,Arroyo Leyes,Arroyo Leyes,Santa Fe



Vista previa MINIBUSES (5 filas):


,LOCALIDAD DE ORIGEN,COORDENADAS ORIGEN,LOCALIDAD DE DESTINO,COORDENADAS DESTINO,PARADAS INTERMEDIAS,DIAS OPERATIVOS,OBSERVACIONES,PARADAS_GEO,PARADAS INTERMEDIAS (CORREGIDAS),LAT_ORIGEN,LON_ORIGEN,LAT_DESTINO,LON_DESTINO
0,Rafaela,"31°14'39.2""S 61°30'02.7""W",Santa Fe,"31°38'00.1""S 60°42'55.2""W",No aplica,"Lunes, Martes, Miercoles, Jueves, Viernes",,No aplica,No aplica,-31.244.222,-61.500.750,-31.633.361,-60.715.333
1,Frontera,"31°27'15.5""S 62°03'13.4""W",Santa Fe,"31°38'00.1""S 60°42'55.2""W","Josefina, María Juana, San Vicente","Lunes, Martes, Miercoles, Jueves, Viernes",,Josefina; María Juana; San Vicente,Josefina; María Juana; San Vicente,-31.454.306,-62.053.722,-31.633.361,-60.715.333
2,Frontera,"31°27'15.5""S 62°03'13.4""W",Rafaela,"31°14'39.2""S 61°30'02.7""W","Josefina, María Juana, San Vicente","Lunes, Martes, Miercoles, Jueves, Viernes",,Josefina; María Juana; San Vicente,Josefina; María Juana; San Vicente,-31.454.306,-62.053.722,-31.244.222,-61.500.750
3,Avellaneda,"29°11'19.0""S 59°41'02.3""W",Santa Fe,"31°38'00.1""S 60°42'55.2""W","Reconquista, Los Laureles, Romang, Alejandra","Lunes, Martes, Miercoles, Jueves, Viernes",semanas alternas,Reconquista; Los Laureles; Romang; Alejandra,Reconquista; Los Laureles; Romang; Alejandra,-29.188.611,-59.683.972,-31.633.361,-60.715.333
4,Florencia,"28°02'44.5""S 59°13'09.7""W",Reconquista,"29°09'26.6""S 59°39'28.8""W","Las Toscas, Villa Ocampo, Guillermina","Lunes, Miercoles, Viernes",,Las Toscas; Villa Ocampo; -28.240306|-59.44800...,Las Toscas; Villa Ocampo; Villa Guillermina,-28.045.694,-59.219.361,-29.157.389,-59.658.000



Listo: gaz_df=(389, 7), mini_df=(26, 13)


In [ ]:
# === CELDA 3E: limpiar puntos de miles en centroide_* y rearmar índice ===
import re, unicodedata, numpy as np, pandas as pd

def _norm(s: str) -> str:
    s = str(s or "").strip().lower()
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c)!="Mn")
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def _normalize_coord(v, bound):
    """Divide entre 10 sucesivamente hasta quedar en rango."""
    try: vv = float(v)
    except: return np.nan
    steps = 0
    while abs(vv) > bound and steps < 20:
        vv /= 10.0
        steps += 1
    return vv if abs(vv) <= bound else np.nan

_thousand_dots_re = re.compile(r"^-?\d{1,3}(?:\.\d{3})+$")  # ej: -312.142.150.140.904

def _clean_thousand_dots(x: str) -> str:
    s = str(x or "").strip()
    if _thousand_dots_re.match(s):
        # quitar todos los puntos (son separadores de miles)
        s = s.replace(".", "")
    return s

def _to_float_relaxed(x):
    """Primero limpia separadores de miles con punto; luego intenta float (también con coma)."""
    s = _clean_thousand_dots(x)
    # si quedó con comas decimales, cambiarlas por punto
    s = s.replace(",", ".")
    try:
        return float(s)
    except:
        return np.nan

# Parsers complementarios por si vinieran en otros formatos (WKT/pares/DMS)
num_re = r"-?\d+(?:[.,]\d+)?"
pair_bracket = re.compile(rf"[\[\(]?\s*({num_re})\s*[, ]\s*({num_re})\s*[\]\)]?")
wkt_point   = re.compile(rf"POINT\s*\(\s*({num_re})\s+({num_re})\s*\)", re.I)

def dms_to_decimal(dms_str):
    if not isinstance(dms_str, str): return (np.nan, np.nan)
    s = dms_str.strip()
    pat = re.compile(
        r"""^\s*
        (?P<lat_deg>\d{1,3})[°:\s]\s*(?P<lat_min>\d{1,2})['’:\s]\s*(?P<lat_sec>\d{1,2}(?:\.\d+)?)["”]?\s*(?P<lat_dir>[NSns])
        [,\s]+
        (?P<lon_deg>\d{1,3})[°:\s]\s*(?P<lon_min>\d{1,2})['’:\s]\s*(?P<lon_sec>\d{1,2}(?:\.\d+)?)["”]?\s*(?P<lon_dir>[EOWeow])
        \s*$""", re.X)
    m = pat.match(s)
    if not m: return (np.nan, np.nan)
    def comp(d, m, s): return float(d) + float(m)/60.0 + float(s)/3600.0
    lat = comp(m['lat_deg'], m['lat_min'], m['lat_sec'])
    lon = comp(m['lon_deg'], m['lon_min'], m['lon_sec'])
    if m['lat_dir'].upper() == 'S': lat *= -1
    if m['lon_dir'].upper() in ('W','O'): lon *= -1
    return (lat, lon)

def parse_centroid_row(row):
    a_raw = row.get("centroide_lat")
    b_raw = row.get("centroide_lon")

    # Caso principal: vienen con “puntos de miles” → limpiar, convertir y normalizar
    a = _to_float_relaxed(a_raw); b = _to_float_relaxed(b_raw)
    la = _normalize_coord(a, 90); lo = _normalize_coord(b, 180)
    if not (np.isnan(la) or np.isnan(lo)):
        # Si pareciera invertido por rangos, intento swap
        if (abs(la) > 90 and abs(lo) <= 90):
            la2 = _normalize_coord(b, 90); lo2 = _normalize_coord(a, 180)
            if not (np.isnan(la2) or np.isnan(lo2)):
                return la2, lo2
        return la, lo

    # Fallbacks (por si algún registro raro vino en otro formato):
    for raw in (a_raw, b_raw):
        s = str(raw or "")
        m = wkt_point.search(s)
        if m:
            lon = _to_float_relaxed(m.group(1)); lat = _to_float_relaxed(m.group(2))
            lat = _normalize_coord(lat, 90); lon = _normalize_coord(lon, 180)
            if abs(lat) > 90 and abs(lon) <= 90:
                lat, lon = lon, lat
            return lat, lon

    for raw in (a_raw, b_raw):
        s = str(raw or "")
        m = pair_bracket.search(s)
        if m:
            x = _to_float_relaxed(m.group(1)); y = _to_float_relaxed(m.group(2))
            lon, lat = x, y
            if abs(lat) > 90 and abs(lon) <= 90:
                lat, lon = lon, lat
            lat = _normalize_coord(lat, 90); lon = _normalize_coord(lon, 180)
            return lat, lon

    for raw in (a_raw, b_raw):
        s = str(raw or "")
        if any(tok in s for tok in ["°","'","\"","N","S","E","O","W","n","s","e","o","w"]):
            la, lo = dms_to_decimal(s)
            la = _normalize_coord(la, 90); lo = _normalize_coord(lo, 180)
            if not (np.isnan(la) or np.isnan(lo)): return la, lo

    return (np.nan, np.nan)

# --- Aplicar al gazetteer ---
gdf2 = gaz_df.copy()
lat_list=[]; lon_list=[]
for _, r in gdf2.iterrows():
    la, lo = parse_centroid_row(r)
    lat_list.append(la); lon_list.append(lo)
gdf2["LAT"] = lat_list; gdf2["LON"] = lon_list

valid = gdf2["LAT"].notna() & gdf2["LON"].notna() & (gdf2["LAT"].abs()<=90) & (gdf2["LON"].abs()<=180)
gdf_ok2 = gdf2.loc[valid].reset_index(drop=True)

print(f"Filas con LAT/LON válidas (tras limpieza): {len(gdf_ok2)} / {len(gdf2)}")
print("\nEjemplos válidos (hasta 5):")
display(gdf_ok2[["nombre","localidad_censal_nombre","municipio_nombre","departamento_nombre","LAT","LON"]].head(5))

# --- Reconstruir índice con el gazetteer limpio ---
gaz_idx = {}
def _add_key(k, la, lo):
    if k and k not in gaz_idx:
        gaz_idx[k] = (float(la), float(lo))

if len(gdf_ok2):
    for _, r in gdf_ok2.iterrows():
        la, lo = r["LAT"], r["LON"]
        base_names=set()
        for c in ["nombre","localidad_censal_nombre","municipio_nombre"]:
            v=r.get(c)
            if isinstance(v,str) and v.strip(): base_names.add(v.strip())
        if not base_names and isinstance(r.get("localidad_censal_nombre"), str):
            v=r.get("localidad_censal_nombre")
            if v and v.strip(): base_names.add(v.strip())
        for nm in base_names:
            _add_key(_norm(nm), la, lo)
        dpto = r.get("departamento_nombre")
        if isinstance(dpto, str) and dpto.strip():
            d=_norm(dpto)
            for nm in base_names:
                _add_key(_norm(nm)+"|"+d, la, lo)

print(f"\n✅ Índice reconstruido: {len(gaz_idx)} claves.")

# --- Re-diagnóstico de paradas en df_full ---
def _parada_ok(p):
    la, lo = p.get("lat"), p.get("lon")
    if (la is not None) and (lo is not None): return True
    nm = _norm(str(p.get("name","")))
    return nm in gaz_idx

paradas_total = df_full["__PARADAS_LIST"].apply(lambda L: len(L) if isinstance(L, list) else 0).sum()
paradas_ok = df_full["__PARADAS_LIST"].apply(
    lambda L: sum(_parada_ok(p) for p in (L if isinstance(L, list) else []))
).sum()
print(f"— RE-DIAGNÓSTICO — Paradas total: {paradas_total} | resueltas (coord/lookup): {paradas_ok} | pendientes: {paradas_total - paradas_ok}")

Filas con LAT/LON válidas (tras limpieza): 389 / 389

Ejemplos válidos (hasta 5):


,nombre,localidad_censal_nombre,municipio_nombre,departamento_nombre,LAT,LON
0,Presidente Roca,Presidente Roca,Presidente Roca,Castellanos,-31.214215,-61.612913
1,Estación Presidente Roca,Presidente Roca,Presidente Roca,Castellanos,-31.232128,-61.610277
2,Villa Adelina,Sauce Viejo,Sauce Viejo,La Capital,-31.721030,-60.801589
3,Sauce Viejo,Sauce Viejo,Sauce Viejo,La Capital,-31.766037,-60.830560
4,Arroyo Leyes,Arroyo Leyes,Arroyo Leyes,La Capital,-31.560445,-60.519179



✅ Índice reconstruido: 787 claves.
— RE-DIAGNÓSTICO — Paradas total: 117 | resueltas (coord/lookup): 113 | pendientes: 4


Celda 3 — Normalización + índice geográfico + parseo de paradas

In [3]:
# === CELDA 3: normalizar gazetteer, construir índice y preparar coordenadas ===
import re, unicodedata, numpy as np, pandas as pd

# ---------- Helpers ----------
def _norm(s: str) -> str:
    """Normaliza texto para matching: minúsculas, sin tildes, sin signos."""
    s = str(s or "").strip().lower()
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c)!="Mn")
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def _to_float(x):
    """Convierte '12,34'/'12.34' a float; si no puede, NaN."""
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return np.nan
    s = str(x).strip().replace(",", ".")
    try:
        return float(s)
    except:
        return np.nan

def _normalize_coord(v, bound):
    """Si vienen números gigantes (p.ej. faltaba el punto), divide entre 10 hasta entrar en rango."""
    try:
        vv = float(v)
    except:
        return np.nan
    steps = 0
    while abs(vv) > bound and steps < 15:
        vv /= 10.0
        steps += 1
    return vv if abs(vv) <= bound else np.nan

def dms_to_decimal(dms_str):
    """
    Convierte '31°14'39.2"S 61°30'02.7"W' → (-31.244222, -61.50075)
    Devuelve (lat, lon) en decimales o (NaN, NaN) si no matchea.
    """
    if not isinstance(dms_str, str):
        return (np.nan, np.nan)
    s = dms_str.strip()
    # Dos grupos: LAT DMS + dir, espacio, LON DMS + dir
    pat = re.compile(
        r"""^\s*
        (?P<lat_deg>\d{1,3})[°:\s]\s*(?P<lat_min>\d{1,2})['’:\s]\s*(?P<lat_sec>\d{1,2}(?:\.\d+)?)["”]?\s*(?P<lat_dir>[NSns])
        [,\s]+
        (?P<lon_deg>\d{1,3})[°:\s]\s*(?P<lon_min>\d{1,2})['’:\s]\s*(?P<lon_sec>\d{1,2}(?:\.\d+)?)["”]?\s*(?P<lon_dir>[EOWeow])
        \s*$""",
        re.X
    )
    m = pat.match(s)
    if not m:
        return (np.nan, np.nan)
    def comp(d, m, s):
        return float(d) + float(m)/60.0 + float(s)/3600.0
    lat = comp(m['lat_deg'], m['lat_min'], m['lat_sec'])
    lon = comp(m['lon_deg'], m['lon_min'], m['lon_sec'])
    if m['lat_dir'].upper() == 'S':
        lat *= -1
    if m['lon_dir'].upper() in ('W','O'):
        lon *= -1
    return (lat, lon)

def parse_paradas_geo(texto):
    """
    PARADAS_GEO en formatos:
      - 'Nombre|lat|lon; Otro|lat|lon'
      - 'lat|lon|Nombre; ...'
      - o lista simple 'Nombre1; Nombre2' (sin coords)
    Devuelve lista de dicts: [{'name':..., 'lat':..., 'lon':...}, ...]
    """
    out = []
    if not isinstance(texto, str):
        return out
    for it in [x.strip() for x in texto.split(";") if x.strip()]:
        parts = [p.strip() for p in it.split("|")]
        name, la, lo = None, None, None
        if len(parts) == 3:
            a, b, c = parts
            # Intento 1: a=lat, b=lon, c=nombre
            try:
                la, lo = float(a.replace(",", ".")), float(b.replace(",", "."))
                name = c
            except:
                # Intento 2: a=nombre, b=lat, c=lon
                name = a
                try:
                    la, lo = float(b.replace(",", ".")), float(c.replace(",", "."))
                except:
                    la = lo = None
        else:
            name = it
        try:
            la = float(str(la).replace(",", ".")) if la not in (None,"","NA") else None
            lo = float(str(lo).replace(",", ".")) if lo not in (None,"","NA") else None
        except:
            la = lo = None
        out.append({"name": (name or "").strip(), "lat": la, "lon": lo})
    return out

# ---------- 3.1 Normalizar GAZETTEER y construir índice ----------
assert {"centroide_lat","centroide_lon"}.issubset(gaz_df.columns), "Faltan columnas centroide_* en el gazetteer."

gdf = gaz_df.copy()
gdf["LAT"] = gdf["centroide_lat"].apply(_to_float).apply(lambda v: _normalize_coord(v, 90))
gdf["LON"] = gdf["centroide_lon"].apply(_to_float).apply(lambda v: _normalize_coord(v, 180))
gdf = gdf.dropna(subset=["LAT","LON"]).reset_index(drop=True)

gaz_idx = {}  # dict: clave_normalizada -> (lat, lon)

def _add_key(k, la, lo):
    if k and k not in gaz_idx:
        gaz_idx[k] = (float(la), float(lo))

name_cols = [c for c in ["nombre","localidad_censal_nombre","municipio_nombre"] if c in gdf.columns]

for _, r in gdf.iterrows():
    la, lo = r["LAT"], r["LON"]
    cand = set()
    for f in name_cols:
        v = r.get(f)
        if isinstance(v, str) and v.strip():
            cand.add(_norm(v))
    dpto = r.get("departamento_nombre")
    if isinstance(dpto, str) and dpto.strip():
        d = _norm(dpto)
        for f in name_cols:
            v = r.get(f)
            if isinstance(v, str) and v.strip():
                cand.add(_norm(v) + "|" + d)
    for k in cand:
        _add_key(k, la, lo)

print(f"✅ Índice geográfico listo: {len(gaz_idx)} claves de búsqueda.")
ej = [k for k in gaz_idx.keys() if "|" in k][:8]
print("   Ejemplos con depto:", ej if ej else "(sin ejemplos)")

# ---------- 3.2 Preparar DF de minibuses y parsear paradas ----------
df = mini_df.copy()

# Paradas: de PARADAS_GEO si existe, si no de PARADAS INTERMEDIAS (lista simple)
if "__PARADAS_LIST" not in df.columns:
    if "PARADAS_GEO" in df.columns:
        df["__PARADAS_LIST"] = df["PARADAS_GEO"].apply(parse_paradas_geo)
    else:
        def _parse_plain(s):
            out=[]
            if not isinstance(s, str): return out
            for p in re.split(r"[;|,/\\\n]+", s):
                p2 = p.strip()
                if p2:
                    out.append({"name": p2, "lat": None, "lon": None})
            return out
        df["__PARADAS_LIST"] = df.get("PARADAS INTERMEDIAS", pd.Series([""]*len(df))).apply(_parse_plain)

# ---------- 3.3 Generar/Rellenar LAT/LON de ORIGEN y DESTINO ----------
# 1) Si ya están, convertir a float y normalizar; 2) si hay DMS en COORDENADAS ORIGEN/DESTINO, parsear;
# 3) si aún faltan, lookup por nombre en el gazetteer.

# ORIGEN
if "LAT_ORIGEN" not in df.columns: df["LAT_ORIGEN"] = np.nan
if "LON_ORIGEN" not in df.columns: df["LON_ORIGEN"] = np.nan
df["LAT_ORIGEN"] = df["LAT_ORIGEN"].apply(_to_float).apply(lambda v: _normalize_coord(v, 90))
df["LON_ORIGEN"] = df["LON_ORIGEN"].apply(_to_float).apply(lambda v: _normalize_coord(v, 180))

if "COORDENADAS ORIGEN" in df.columns:
    mask_o = df["LAT_ORIGEN"].isna() | df["LON_ORIGEN"].isna()
    latlon_o = df.loc[mask_o, "COORDENADAS ORIGEN"].apply(dms_to_decimal)
    df.loc[mask_o, "LAT_ORIGEN"] = [p[0] for p in latlon_o]
    df.loc[mask_o, "LON_ORIGEN"] = [p[1] for p in latlon_o]

mask_o = df["LAT_ORIGEN"].isna() | df["LON_ORIGEN"].isna()
if "LOCALIDAD DE ORIGEN" in df.columns:
    def _lookup_lat(row):
        if not mask_o.loc[row.name]: return row["LAT_ORIGEN"]
        k = _norm(row.get("LOCALIDAD DE ORIGEN",""))
        return gaz_idx.get(k, (np.nan, np.nan))[0]
    def _lookup_lon(row):
        if not mask_o.loc[row.name]: return row["LON_ORIGEN"]
        k = _norm(row.get("LOCALIDAD DE ORIGEN",""))
        return gaz_idx.get(k, (np.nan, np.nan))[1]
    df["LAT_ORIGEN"] = df.apply(_lookup_lat, axis=1)
    df["LON_ORIGEN"] = df.apply(_lookup_lon, axis=1)

# DESTINO
if "LAT_DESTINO" not in df.columns: df["LAT_DESTINO"] = np.nan
if "LON_DESTINO" not in df.columns: df["LON_DESTINO"] = np.nan
df["LAT_DESTINO"] = df["LAT_DESTINO"].apply(_to_float).apply(lambda v: _normalize_coord(v, 90))
df["LON_DESTINO"] = df["LON_DESTINO"].apply(_to_float).apply(lambda v: _normalize_coord(v, 180))

if "COORDENADAS DESTINO" in df.columns:
    mask_d = df["LAT_DESTINO"].isna() | df["LON_DESTINO"].isna()
    latlon_d = df.loc[mask_d, "COORDENADAS DESTINO"].apply(dms_to_decimal)
    df.loc[mask_d, "LAT_DESTINO"] = [p[0] for p in latlon_d]
    df.loc[mask_d, "LON_DESTINO"] = [p[1] for p in latlon_d]

mask_d = df["LAT_DESTINO"].isna() | df["LON_DESTINO"].isna()
if "LOCALIDAD DE DESTINO" in df.columns:
    def _lookup_lat_d(row):
        if not mask_d.loc[row.name]: return row["LAT_DESTINO"]
        k = _norm(row.get("LOCALIDAD DE DESTINO",""))
        return gaz_idx.get(k, (np.nan, np.nan))[0]
    def _lookup_lon_d(row):
        if not mask_d.loc[row.name]: return row["LON_DESTINO"]
        k = _norm(row.get("LOCALIDAD DE DESTINO",""))
        return gaz_idx.get(k, (np.nan, np.nan))[1]
    df["LAT_DESTINO"] = df.apply(_lookup_lat_d, axis=1)
    df["LON_DESTINO"] = df.apply(_lookup_lon_d, axis=1)

# ---------- 3.4 Conteos y muestras ----------
n_o = df["LAT_ORIGEN"].notna().sum()
n_d = df["LAT_DESTINO"].notna().sum()

# Paradas con coords explícitas o por gazetteer
def _parada_ok(p):
    la, lo = p.get("lat"), p.get("lon")
    if la is not None and lo is not None:
        return True
    # si no hay lat/lon, pruebo en gaz_idx
    nm = _norm(str(p.get("name","")))
    return nm in gaz_idx

paradas_total = df["__PARADAS_LIST"].apply(lambda L: len(L) if isinstance(L, list) else 0).sum()
paradas_ok = df["__PARADAS_LIST"].apply(
    lambda L: sum(_parada_ok(p) for p in (L if isinstance(L, list) else []))
).sum()
paradas_pend = paradas_total - paradas_ok

print("\n— RESUMEN —")
print(f"Orígenes con lat/lon: {n_o} / {len(df)}")
print(f"Destinos con lat/lon: {n_d} / {len(df)}")
print(f"Paradas (total items): {paradas_total} | con coord/lookup: {paradas_ok} | pendientes: {paradas_pend}")

print("\nMuestra ORIGEN (5 filas):")
display(df[["LOCALIDAD DE ORIGEN","LAT_ORIGEN","LON_ORIGEN"]].head(5))
print("\nMuestra DESTINO (5 filas):")
display(df[["LOCALIDAD DE DESTINO","LAT_DESTINO","LON_DESTINO"]].head(5))

# Guardamos como 'df_full' para las siguientes celdas
df_full = df.copy()


✅ Índice geográfico listo: 0 claves de búsqueda.
   Ejemplos con depto: (sin ejemplos)

— RESUMEN —
Orígenes con lat/lon: 25 / 26
Destinos con lat/lon: 26 / 26
Paradas (total items): 117 | con coord/lookup: 14 | pendientes: 103

Muestra ORIGEN (5 filas):


,LOCALIDAD DE ORIGEN,LAT_ORIGEN,LON_ORIGEN
0,Rafaela,-31.244222,-61.500750
1,Frontera,-31.454306,-62.053722
2,Frontera,-31.454306,-62.053722
3,Avellaneda,-29.188611,-59.683972
4,Florencia,-28.045694,-59.219361



Muestra DESTINO (5 filas):


,LOCALIDAD DE DESTINO,LAT_DESTINO,LON_DESTINO
0,Santa Fe,-31.633361,-60.715333
1,Santa Fe,-31.633361,-60.715333
2,Rafaela,-31.244222,-61.500750
3,Santa Fe,-31.633361,-60.715333
4,Reconquista,-29.157389,-59.658000


In [4]:
# === CELDA 3C: chequear columnas y reconstruir índice del gazetteer ===
import pandas as pd, numpy as np, re, unicodedata

# Usamos gaz_df (de Celda 2)
assert isinstance(gaz_df, pd.DataFrame), "No encuentro gaz_df. Corré Celda 2."

def _norm(s: str) -> str:
    s = str(s or "").strip().lower()
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c)!="Mn")
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def _to_float(x):
    if x is None or (isinstance(x, float) and np.isnan(x)): return np.nan
    s = str(x).strip().replace(",", ".")
    try: return float(s)
    except: return np.nan

def _normalize_coord(v, bound):
    try: vv=float(v)
    except: return np.nan
    steps=0
    while abs(vv)>bound and steps<15:
        vv/=10.0; steps+=1
    return vv if abs(vv)<=bound else np.nan

# 1) Controles de presencia y no-nulos por columna clave
cols_esperadas = ["centroide_lat","centroide_lon","departamento_nombre","localidad_censal_nombre","municipio_nombre","nombre","provincia_nombre"]
presentes = [c for c in cols_esperadas if c in gaz_df.columns]
faltan    = [c for c in cols_esperadas if c not in gaz_df.columns]
print("Columnas presentes:", presentes)
print("Columnas faltantes:", faltan or "—")

def nn(col):
    return gaz_df[col].astype(str).str.strip().replace({"":np.nan}).notna().sum() if col in gaz_df.columns else 0

for c in ["nombre","localidad_censal_nombre","municipio_nombre","departamento_nombre","provincia_nombre"]:
    print(f"• No vacíos en {c:>24}: {nn(c)}")

# 2) Normalizar lat/lon y opcionalmente filtrar provincia (si hubiera otras)
gdf = gaz_df.copy()
gdf["LAT"] = gdf["centroide_lat"].apply(_to_float).apply(lambda v:_normalize_coord(v,90))
gdf["LON"] = gdf["centroide_lon"].apply(_to_float).apply(lambda v:_normalize_coord(v,180))
before = len(gdf)
gdf = gdf.dropna(subset=["LAT","LON"]).reset_index(drop=True)
print(f"\nFilas con LAT/LON válidas: {len(gdf)} / {before}")

# Si hay varias provincias, quedate con Santa Fe (si existe)
if "provincia_nombre" in gdf.columns:
    provs = sorted(set(str(x).strip() for x in gdf["provincia_nombre"].dropna()))
    print("Provincias detectadas:", provs)
    if len(provs) > 1 and "Santa Fe" in provs:
        gdf = gdf[gdf["provincia_nombre"].astype(str).str.strip()=="Santa Fe"].reset_index(drop=True)
        print("↳ Filtrado a provincia = Santa Fe → filas:", len(gdf))

# 3) Reconstruir índice de claves → (lat,lon)
gaz_idx = {}
def _add_key(k, la, lo):
    if k and k not in gaz_idx:
        gaz_idx[k] = (float(la), float(lo))

for _, r in gdf.iterrows():
    la, lo = r["LAT"], r["LON"]
    # nombres disponibles (usamos TODO lo que traiga algo de texto)
    base_names = set()
    for c in ["nombre","localidad_censal_nombre","municipio_nombre"]:
        if c in gdf.columns:
            v = r.get(c)
            if isinstance(v, str) and v.strip():
                base_names.add(v.strip())
    # si no hay 'nombre', forzamos 'localidad_censal_nombre' si existe
    if not base_names and "localidad_censal_nombre" in gdf.columns:
        v = r.get("localidad_censal_nombre")
        if isinstance(v, str) and v.strip():
            base_names.add(v.strip())

    # agregar simples
    for nm in base_names:
        _add_key(_norm(nm), la, lo)

    # agregar variantes con departamento
    dpto = r.get("departamento_nombre")
    if isinstance(dpto, str) and dpto.strip():
        d = _norm(dpto)
        for nm in base_names:
            _add_key(_norm(nm) + "|" + d, la, lo)

print(f"\n✅ Índice reconstruido: {len(gaz_idx)} claves.")
ej_simples = [k for k in gaz_idx if "|" not in k][:8]
ej_comp    = [k for k in gaz_idx if "|" in k][:8]
print("   Ejemplos simples:", ej_simples or "(—)")
print("   Ejemplos con depto:", ej_comp or "(—)")

# 4) Re-diagnóstico de paradas con el nuevo índice (usa df_full de la Celda 3)
def _parada_ok(p):
    la, lo = p.get("lat"), p.get("lon")
    if (la is not None) and (lo is not None):
        return True
    nm = _norm(str(p.get("name","")))
    return nm in gaz_idx

paradas_total = df_full["__PARADAS_LIST"].apply(lambda L: len(L) if isinstance(L, list) else 0).sum()
paradas_ok = df_full["__PARADAS_LIST"].apply(
    lambda L: sum(_parada_ok(p) for p in (L if isinstance(L, list) else []))
).sum()
print(f"\n— RE-DIAGNÓSTICO — Paradas total: {paradas_total} | resueltas (coord/lookup): {paradas_ok} | pendientes: {paradas_total - paradas_ok}")

# 5) Muestreo de hasta 12 paradas y si están en el índice
uniq_paradas = []
for L in df_full["__PARADAS_LIST"]:
    if isinstance(L, list):
        for p in L:
            nm = str(p.get("name","")).strip()
            if nm and nm not in uniq_paradas:
                uniq_paradas.append(nm)
    if len(uniq_paradas) >= 12:
        break

def lookup_row(nm):
    k = _norm(nm)
    xy = gaz_idx.get(k)
    return {"parada": nm, "en_gazetteer": bool(xy), "lat": (xy[0] if xy else np.nan), "lon": (xy[1] if xy else np.nan)}

test_df = pd.DataFrame([lookup_row(nm) for nm in uniq_paradas])
print("\nMuestreo (hasta 12) → presencia en gazetteer:")
display(test_df)


Columnas presentes: ['centroide_lat', 'centroide_lon', 'departamento_nombre', 'localidad_censal_nombre', 'municipio_nombre', 'nombre', 'provincia_nombre']
Columnas faltantes: —
• No vacíos en                   nombre: 389
• No vacíos en  localidad_censal_nombre: 389
• No vacíos en         municipio_nombre: 389
• No vacíos en      departamento_nombre: 389
• No vacíos en         provincia_nombre: 389

Filas con LAT/LON válidas: 0 / 389
Provincias detectadas: []

✅ Índice reconstruido: 0 claves.
   Ejemplos simples: (—)
   Ejemplos con depto: (—)

— RE-DIAGNÓSTICO — Paradas total: 117 | resueltas (coord/lookup): 14 | pendientes: 103

Muestreo (hasta 12) → presencia en gazetteer:


,parada,en_gazetteer,lat,lon
0,No aplica,False,NaN,NaN
1,Josefina,False,NaN,NaN
2,María Juana,False,NaN,NaN
3,San Vicente,False,NaN,NaN
4,Reconquista,False,NaN,NaN
5,Los Laureles,False,NaN,NaN
6,Romang,False,NaN,NaN
7,Alejandra,False,NaN,NaN
8,Las Toscas,False,NaN,NaN
9,Villa Ocampo,False,NaN,NaN


3D

In [5]:
# === CELDA 3D: parseo robusto de coordenadas + índice y diagnóstico ===
import re, unicodedata, numpy as np, pandas as pd

def _norm(s: str) -> str:
    s = str(s or "").strip().lower()
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c)!="Mn")
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def _to_float(x):
    if x is None or (isinstance(x, float) and np.isnan(x)): return np.nan
    s = str(x).strip().replace(",", ".")
    try: return float(s)
    except: return np.nan

def _normalize_coord(v, bound):
    try: vv=float(v)
    except: return np.nan
    steps=0
    while abs(vv)>bound and steps<15:
        vv/=10.0; steps+=1
    return vv if abs(vv)<=bound else np.nan

def dms_to_decimal(dms_str):
    if not isinstance(dms_str, str): return (np.nan, np.nan)
    s = dms_str.strip()
    pat = re.compile(
        r"""^\s*
        (?P<lat_deg>\d{1,3})[°:\s]\s*(?P<lat_min>\d{1,2})['’:\s]\s*(?P<lat_sec>\d{1,2}(?:\.\d+)?)["”]?\s*(?P<lat_dir>[NSns])
        [,\s]+
        (?P<lon_deg>\d{1,3})[°:\s]\s*(?P<lon_min>\d{1,2})['’:\s]\s*(?P<lon_sec>\d{1,2}(?:\.\d+)?)["”]?\s*(?P<lon_dir>[EOWeow])
        \s*$""",
        re.X
    )
    m = pat.match(s)
    if not m: return (np.nan, np.nan)
    def comp(d, m, s): return float(d) + float(m)/60.0 + float(s)/3600.0
    lat = comp(m['lat_deg'], m['lat_min'], m['lat_sec'])
    lon = comp(m['lon_deg'], m['lon_min'], m['lon_sec'])
    if m['lat_dir'].upper() == 'S': lat *= -1
    if m['lon_dir'].upper() in ('W','O'): lon *= -1
    return (lat, lon)

# --- 1) Inspección rápida de formatos en tu hoja ---
def sample_vals(series, n=8):
    vals=[]
    for v in series.head(50):
        if v not in vals and str(v).strip():
            vals.append(v)
        if len(vals)>=n: break
    return vals

print("Muestras crudas centroide_lat:", sample_vals(gaz_df["centroide_lat"]))
print("Muestras crudas centroide_lon:", sample_vals(gaz_df["centroide_lon"]))

# --- 2) Parser robusto por fila: puede leer (lat,lon) desde múltiples formatos ---
num_re = r"-?\d+(?:[.,]\d+)?"
pair_bracket = re.compile(rf"[\[\(]?\s*({num_re})\s*[, ]\s*({num_re})\s*[\]\)]?")
wkt_point   = re.compile(rf"POINT\s*\(\s*({num_re})\s+({num_re})\s*\)", re.I)

def parse_centroid_row(row):
    a_raw = row.get("centroide_lat")
    b_raw = row.get("centroide_lon")

    # Caso 1: ambos parecen decimales sueltos
    a = _to_float(a_raw); b = _to_float(b_raw)
    if not np.isnan(a) and not np.isnan(b):
        la = _normalize_coord(a, 90); lo = _normalize_coord(b, 180)
        # si quedaron invertidos (lat fuera de rango y lon dentro), intentá swap
        if (np.isnan(la) or abs(la)>90) and (not np.isnan(lo) and abs(lo)<=90):
            la2 = _normalize_coord(b, 90); lo2 = _normalize_coord(a, 180)
            if not np.isnan(la2) and not np.isnan(lo2): return la2, lo2
        return la, lo

    # Caso 2: alguno trae WKT "POINT(lon lat)"
    for raw in (a_raw, b_raw):
        s = str(raw or "")
        m = wkt_point.search(s)
        if m:
            lon_s, lat_s = m.group(1), m.group(2)
            lon = _to_float(lon_s); lat = _to_float(lat_s)
            lat = _normalize_coord(lat, 90); lon = _normalize_coord(lon, 180)
            # si por orden vienen al revés, detecto por rangos
            if abs(lat)>90 and abs(lon)<=90:
                lat, lon = lon, lat
            return lat, lon

    # Caso 3: alguno trae par [lon, lat] o (lon, lat)
    for raw in (a_raw, b_raw):
        s = str(raw or "")
        m = pair_bracket.search(s)
        if m:
            x = _to_float(m.group(1)); y = _to_float(m.group(2))
            # suposición común: [lon, lat]
            lon, lat = x, y
            # chequeo rangos
            if abs(lat)>90 and abs(lon)<=90:
                lat, lon = lon, lat
            lat = _normalize_coord(lat, 90); lon = _normalize_coord(lon, 180)
            return lat, lon

    # Caso 4: uno de los dos está en DMS combinado (raro)
    for raw in (a_raw, b_raw):
        s = str(raw or "")
        if any(tok in s for tok in ["°","'","\"","N","S","E","O","W","n","s","e","o","w"]):
            la, lo = dms_to_decimal(s)
            if not (np.isnan(la) or np.isnan(lo)):
                return _normalize_coord(la,90), _normalize_coord(lo,180)

    # Último intento: números gigantes -> normalizo
    a = _normalize_coord(_to_float(a_raw), 90)
    b = _normalize_coord(_to_float(b_raw), 180)
    return a, b

# --- 3) Aplicar parser y medir cobertura ---
gdf = gaz_df.copy()
lat_list=[]; lon_list=[]
for _, r in gdf.iterrows():
    la, lo = parse_centroid_row(r)
    lat_list.append(la); lon_list.append(lo)
gdf["LAT"] = lat_list; gdf["LON"] = lon_list

valid = gdf["LAT"].notna() & gdf["LON"].notna() & (gdf["LAT"].abs()<=90) & (gdf["LON"].abs()<=180)
gdf_ok = gdf.loc[valid].reset_index(drop=True)
print(f"\nFilas con LAT/LON válidas (post-parser): {len(gdf_ok)} / {len(gdf)}")

print("\nEjemplos válidos (hasta 5):")
display(gdf_ok[["nombre","localidad_censal_nombre","municipio_nombre","departamento_nombre","LAT","LON"]].head(5))

# --- 4) Reconstruir índice si hay filas válidas ---
gaz_idx = {}
def _add_key(k, la, lo):
    if k and k not in gaz_idx: gaz_idx[k]=(float(la), float(lo))

if len(gdf_ok):
    for _, r in gdf_ok.iterrows():
        la, lo = r["LAT"], r["LON"]
        base_names=set()
        for c in ["nombre","localidad_censal_nombre","municipio_nombre"]:
            if c in r.index:
                v=r.get(c)
                if isinstance(v,str) and v.strip(): base_names.add(v.strip())
        if not base_names and isinstance(r.get("localidad_censal_nombre"), str):
            v=r.get("localidad_censal_nombre")
            if v and v.strip(): base_names.add(v.strip())
        for nm in base_names:
            _add_key(_norm(nm), la, lo)
        dpto = r.get("departamento_nombre")
        if isinstance(dpto, str) and dpto.strip():
            d=_norm(dpto)
            for nm in base_names:
                _add_key(_norm(nm)+"|"+d, la, lo)

print(f"✅ Índice reconstruido: {len(gaz_idx)} claves.")

# --- 5) Re-diagnóstico de paradas en df_full ---
def _parada_ok(p):
    la, lo = p.get("lat"), p.get("lon")
    if (la is not None) and (lo is not None): return True
    nm = _norm(str(p.get("name","")))
    return nm in gaz_idx

paradas_total = df_full["__PARADAS_LIST"].apply(lambda L: len(L) if isinstance(L, list) else 0).sum()
paradas_ok = df_full["__PARADAS_LIST"].apply(
    lambda L: sum(_parada_ok(p) for p in (L if isinstance(L, list) else []))
).sum()
print(f"— RE-DIAGNÓSTICO — Paradas total: {paradas_total} | resueltas (coord/lookup): {paradas_ok} | pendientes: {paradas_total - paradas_ok}")

# Muestreo de presencia en índice
uniq_paradas=[]
for L in df_full["__PARADAS_LIST"]:
    if isinstance(L, list):
        for p in L:
            nm=str(p.get("name","")).strip()
            if nm and nm not in uniq_paradas:
                uniq_paradas.append(nm)
    if len(uniq_paradas)>=12: break

def lookup_row(nm):
    k=_norm(nm); xy=gaz_idx.get(k)
    return {"parada": nm, "en_gazetteer": bool(xy),
            "lat": (xy[0] if xy else np.nan), "lon": (xy[1] if xy else np.nan)}

test_df = pd.DataFrame([lookup_row(nm) for nm in uniq_paradas])
print("\nMuestreo (hasta 12) → presencia en gazetteer:")
display(test_df)


Muestras crudas centroide_lat: ['-312.142.150.140.904', '-312.321.284.987.409', '-317.210.301.786.451', '-317.660.371.876.277', '-315.604.450.839.944', '-315.718.361.103.601', '-324.236.315.024.218', '-324.750.508.722.115']
Muestras crudas centroide_lon: ['-616.129.125.462.879', '-616.102.774.667.774', '-608.015.887.552.447', '-608.305.595.552.308', '-605.191.788.520.517', '-605.310.909.581.548', '-618.897.360.595.799', '-615.765.081.663.526']

Filas con LAT/LON válidas (post-parser): 0 / 389

Ejemplos válidos (hasta 5):


,nombre,localidad_censal_nombre,municipio_nombre,departamento_nombre,LAT,LON


✅ Índice reconstruido: 0 claves.
— RE-DIAGNÓSTICO — Paradas total: 117 | resueltas (coord/lookup): 14 | pendientes: 103

Muestreo (hasta 12) → presencia en gazetteer:


,parada,en_gazetteer,lat,lon
0,No aplica,False,NaN,NaN
1,Josefina,False,NaN,NaN
2,María Juana,False,NaN,NaN
3,San Vicente,False,NaN,NaN
4,Reconquista,False,NaN,NaN
5,Los Laureles,False,NaN,NaN
6,Romang,False,NaN,NaN
7,Alejandra,False,NaN,NaN
8,Las Toscas,False,NaN,NaN
9,Villa Ocampo,False,NaN,NaN


In [6]:
# === CELDA 3E: limpiar puntos de miles en centroide_* y rearmar índice ===
import re, unicodedata, numpy as np, pandas as pd

def _norm(s: str) -> str:
    s = str(s or "").strip().lower()
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c)!="Mn")
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def _normalize_coord(v, bound):
    """Divide entre 10 sucesivamente hasta quedar en rango."""
    try: vv = float(v)
    except: return np.nan
    steps = 0
    while abs(vv) > bound and steps < 20:
        vv /= 10.0
        steps += 1
    return vv if abs(vv) <= bound else np.nan

_thousand_dots_re = re.compile(r"^-?\d{1,3}(?:\.\d{3})+$")  # ej: -312.142.150.140.904

def _clean_thousand_dots(x: str) -> str:
    s = str(x or "").strip()
    if _thousand_dots_re.match(s):
        # quitar todos los puntos (son separadores de miles)
        s = s.replace(".", "")
    return s

def _to_float_relaxed(x):
    """Primero limpia separadores de miles con punto; luego intenta float (también con coma)."""
    s = _clean_thousand_dots(x)
    # si quedó con comas decimales, cambiarlas por punto
    s = s.replace(",", ".")
    try:
        return float(s)
    except:
        return np.nan

# Parsers complementarios por si vinieran en otros formatos (WKT/pares/DMS)
num_re = r"-?\d+(?:[.,]\d+)?"
pair_bracket = re.compile(rf"[\[\(]?\s*({num_re})\s*[, ]\s*({num_re})\s*[\]\)]?")
wkt_point   = re.compile(rf"POINT\s*\(\s*({num_re})\s+({num_re})\s*\)", re.I)

def dms_to_decimal(dms_str):
    if not isinstance(dms_str, str): return (np.nan, np.nan)
    s = dms_str.strip()
    pat = re.compile(
        r"""^\s*
        (?P<lat_deg>\d{1,3})[°:\s]\s*(?P<lat_min>\d{1,2})['’:\s]\s*(?P<lat_sec>\d{1,2}(?:\.\d+)?)["”]?\s*(?P<lat_dir>[NSns])
        [,\s]+
        (?P<lon_deg>\d{1,3})[°:\s]\s*(?P<lon_min>\d{1,2})['’:\s]\s*(?P<lon_sec>\d{1,2}(?:\.\d+)?)["”]?\s*(?P<lon_dir>[EOWeow])
        \s*$""", re.X)
    m = pat.match(s)
    if not m: return (np.nan, np.nan)
    def comp(d, m, s): return float(d) + float(m)/60.0 + float(s)/3600.0
    lat = comp(m['lat_deg'], m['lat_min'], m['lat_sec'])
    lon = comp(m['lon_deg'], m['lon_min'], m['lon_sec'])
    if m['lat_dir'].upper() == 'S': lat *= -1
    if m['lon_dir'].upper() in ('W','O'): lon *= -1
    return (lat, lon)

def parse_centroid_row(row):
    a_raw = row.get("centroide_lat")
    b_raw = row.get("centroide_lon")

    # Caso principal: vienen con “puntos de miles” → limpiar, convertir y normalizar
    a = _to_float_relaxed(a_raw); b = _to_float_relaxed(b_raw)
    la = _normalize_coord(a, 90); lo = _normalize_coord(b, 180)
    if not (np.isnan(la) or np.isnan(lo)):
        # Si pareciera invertido por rangos, intento swap
        if (abs(la) > 90 and abs(lo) <= 90):
            la2 = _normalize_coord(b, 90); lo2 = _normalize_coord(a, 180)
            if not (np.isnan(la2) or np.isnan(lo2)):
                return la2, lo2
        return la, lo

    # Fallbacks (por si algún registro raro vino en otro formato):
    for raw in (a_raw, b_raw):
        s = str(raw or "")
        m = wkt_point.search(s)
        if m:
            lon = _to_float_relaxed(m.group(1)); lat = _to_float_relaxed(m.group(2))
            lat = _normalize_coord(lat, 90); lon = _normalize_coord(lon, 180)
            if abs(lat) > 90 and abs(lon) <= 90:
                lat, lon = lon, lat
            return lat, lon

    for raw in (a_raw, b_raw):
        s = str(raw or "")
        m = pair_bracket.search(s)
        if m:
            x = _to_float_relaxed(m.group(1)); y = _to_float_relaxed(m.group(2))
            lon, lat = x, y
            if abs(lat) > 90 and abs(lon) <= 90:
                lat, lon = lon, lat
            lat = _normalize_coord(lat, 90); lon = _normalize_coord(lon, 180)
            return lat, lon

    for raw in (a_raw, b_raw):
        s = str(raw or "")
        if any(tok in s for tok in ["°","'","\"","N","S","E","O","W","n","s","e","o","w"]):
            la, lo = dms_to_decimal(s)
            la = _normalize_coord(la, 90); lo = _normalize_coord(lo, 180)
            if not (np.isnan(la) or np.isnan(lo)): return la, lo

    return (np.nan, np.nan)

# --- Aplicar al gazetteer ---
gdf2 = gaz_df.copy()
lat_list=[]; lon_list=[]
for _, r in gdf2.iterrows():
    la, lo = parse_centroid_row(r)
    lat_list.append(la); lon_list.append(lo)
gdf2["LAT"] = lat_list; gdf2["LON"] = lon_list

valid = gdf2["LAT"].notna() & gdf2["LON"].notna() & (gdf2["LAT"].abs()<=90) & (gdf2["LON"].abs()<=180)
gdf_ok2 = gdf2.loc[valid].reset_index(drop=True)

print(f"Filas con LAT/LON válidas (tras limpieza): {len(gdf_ok2)} / {len(gdf2)}")
print("\nEjemplos válidos (hasta 5):")
display(gdf_ok2[["nombre","localidad_censal_nombre","municipio_nombre","departamento_nombre","LAT","LON"]].head(5))

# --- Reconstruir índice con el gazetteer limpio ---
gaz_idx = {}
def _add_key(k, la, lo):
    if k and k not in gaz_idx:
        gaz_idx[k] = (float(la), float(lo))

if len(gdf_ok2):
    for _, r in gdf_ok2.iterrows():
        la, lo = r["LAT"], r["LON"]
        base_names=set()
        for c in ["nombre","localidad_censal_nombre","municipio_nombre"]:
            v=r.get(c)
            if isinstance(v,str) and v.strip(): base_names.add(v.strip())
        if not base_names and isinstance(r.get("localidad_censal_nombre"), str):
            v=r.get("localidad_censal_nombre")
            if v and v.strip(): base_names.add(v.strip())
        for nm in base_names:
            _add_key(_norm(nm), la, lo)
        dpto = r.get("departamento_nombre")
        if isinstance(dpto, str) and dpto.strip():
            d=_norm(dpto)
            for nm in base_names:
                _add_key(_norm(nm)+"|"+d, la, lo)

print(f"\n✅ Índice reconstruido: {len(gaz_idx)} claves.")

# --- Re-diagnóstico de paradas en df_full ---
def _parada_ok(p):
    la, lo = p.get("lat"), p.get("lon")
    if (la is not None) and (lo is not None): return True
    nm = _norm(str(p.get("name","")))
    return nm in gaz_idx

paradas_total = df_full["__PARADAS_LIST"].apply(lambda L: len(L) if isinstance(L, list) else 0).sum()
paradas_ok = df_full["__PARADAS_LIST"].apply(
    lambda L: sum(_parada_ok(p) for p in (L if isinstance(L, list) else []))
).sum()
print(f"— RE-DIAGNÓSTICO — Paradas total: {paradas_total} | resueltas (coord/lookup): {paradas_ok} | pendientes: {paradas_total - paradas_ok}")


Filas con LAT/LON válidas (tras limpieza): 389 / 389

Ejemplos válidos (hasta 5):


,nombre,localidad_censal_nombre,municipio_nombre,departamento_nombre,LAT,LON
0,Presidente Roca,Presidente Roca,Presidente Roca,Castellanos,-31.214215,-61.612913
1,Estación Presidente Roca,Presidente Roca,Presidente Roca,Castellanos,-31.232128,-61.610277
2,Villa Adelina,Sauce Viejo,Sauce Viejo,La Capital,-31.721030,-60.801589
3,Sauce Viejo,Sauce Viejo,Sauce Viejo,La Capital,-31.766037,-60.830560
4,Arroyo Leyes,Arroyo Leyes,Arroyo Leyes,La Capital,-31.560445,-60.519179



✅ Índice reconstruido: 787 claves.
— RE-DIAGNÓSTICO — Paradas total: 117 | resueltas (coord/lookup): 113 | pendientes: 4


Celda 4 — Detectar paradas pendientes + sugerencias automáticas

Qué hace:

Cuenta las paradas sin coordenadas ni match.

Propone hasta 3 sugerencias por cada una usando coincidencia difusa.

Muestra una tabla ordenada por frecuencia.

In [7]:
# === CELDA 4: pendientes + sugerencias (fuzzy) ===
!pip -q install rapidfuzz
import pandas as pd, numpy as np, re, unicodedata
from rapidfuzz import process, fuzz

# Usa variables de celdas anteriores: df_full (minibuses), gdf_ok2 (gazetteer limpio), gaz_idx, _norm()

# 1) Banco de nombres del gazetteer (para sugerencias)
cand_names = set()
name_cols = [c for c in ["nombre","localidad_censal_nombre","municipio_nombre"] if c in gdf_ok2.columns]
for c in name_cols:
    cand_names.update([str(x).strip() for x in gdf_ok2[c].dropna() if str(x).strip()])

# Variante con departamento (ayuda a desambiguar en el display)
if "departamento_nombre" in gdf_ok2.columns and name_cols:
    for _, r in gdf_ok2.iterrows():
        dpto = str(r.get("departamento_nombre","")).strip()
        for c in name_cols:
            v = r.get(c)
            if isinstance(v,str) and v.strip():
                cand_names.add(f"{v.strip()} ({dpto})")

cand_names = sorted(cand_names)

# 2) Paradas pendientes (sin lat/lon explícitas y sin match en gaz_idx)
pend_counter = {}
def is_pending(p):
    la, lo = p.get("lat"), p.get("lon")
    if la is not None and lo is not None:
        return False
    nm = str(p.get("name","")).strip()
    if not nm:
        return False
    kn = _norm(nm)
    if kn in gaz_idx:
        return False
    if kn in {"no aplica","noaplica","-","n/a"}:
        return False
    return True

for L in df_full["__PARADAS_LIST"]:
    if isinstance(L, list):
        for p in L:
            if is_pending(p):
                nm = str(p.get("name","")).strip()
                pend_counter[nm] = pend_counter.get(nm, 0) + 1

pend_list = sorted(pend_counter.items(), key=lambda x: (-x[1], x[0]))

# 3) Sugerencias fuzzy (top-3)
def suggs(query, k=3):
    # normalizamos ambos lados dentro del scorer
    res = process.extract(
        query, cand_names,
        scorer=lambda a,b: fuzz.WRatio(_norm(a), _norm(b)),
        limit=k,
    )
    # res: [(choice, score, idx), ...]
    return [f"{ch} ({sc})" for ch, sc, _ in res]

rows=[]
for nm, n in pend_list:
    s = suggs(nm, 3)
    rows.append({
        "PARADA": nm,
        "OCURRENCIAS": n,
        "SUG1": s[0] if len(s)>0 else "",
        "SUG2": s[1] if len(s)>1 else "",
        "SUG3": s[2] if len(s)>2 else "",
    })

df_diag = pd.DataFrame(rows, columns=["PARADA","OCURRENCIAS","SUG1","SUG2","SUG3"])
print(f"Total pendientes: {len(df_diag)}")
display(df_diag)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.8 MB/s eta 0:00:00
Total pendientes: 0


,PARADA,OCURRENCIAS,SUG1,SUG2,SUG3


In [8]:
# === CELDA 4 (fix): pendientes + sugerencias usando processor de RapidFuzz ===
import pandas as pd, numpy as np, re, unicodedata
from rapidfuzz import process, fuzz

# --- helpers mínimos (por si no quedaron en memoria) ---
def _norm(s: str) -> str:
    s = str(s or "").strip().lower()
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c)!="Mn")
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def is_pending(p):
    la, lo = p.get("lat"), p.get("lon")
    if la is not None and lo is not None:
        return False
    nm = str(p.get("name","")).strip()
    if not nm:
        return False
    kn = _norm(nm)
    if kn in gaz_idx:
        return False
    if kn in {"no aplica","noaplica","-","n/a"}:
        return False
    return True

# --- banco de candidatos del gazetteer (nombres y variantes con dpto, sólo para sugerir texto) ---
cand_names = set()
name_cols = [c for c in ["nombre","localidad_censal_nombre","municipio_nombre"] if c in gdf_ok2.columns]
for c in name_cols:
    cand_names.update([str(x).strip() for x in gdf_ok2[c].dropna() if str(x).strip()])

if "departamento_nombre" in gdf_ok2.columns and name_cols:
    for _, r in gdf_ok2.iterrows():
        dpto = str(r.get("departamento_nombre","")).strip()
        for c in name_cols:
            v = r.get(c)
            if isinstance(v,str) and v.strip():
                cand_names.add(f"{v.strip()} ({dpto})")

cand_names = sorted(cand_names)

# --- pendientes en las paradas ---
pend_counter = {}
for L in df_full["__PARADAS_LIST"]:
    if isinstance(L, list):
        for p in L:
            if is_pending(p):
                nm = str(p.get("name","")).strip()
                pend_counter[nm] = pend_counter.get(nm, 0) + 1

pend_list = sorted(pend_counter.items(), key=lambda x: (-x[1], x[0]))

# --- sugerencias: usamos processor=_norm y scorer estándar ---
def suggs(query, k=3):
    res = process.extract(
        query,
        cand_names,
        processor=_norm,       # normaliza ambos lados
        scorer=fuzz.WRatio,    # distancia robusta
        limit=k
    )
    return [f"{choice} ({int(score)})" for choice, score, _ in res]

rows=[]
for nm, n in pend_list:
    s = suggs(nm, 3)
    rows.append({
        "PARADA": nm,
        "OCURRENCIAS": n,
        "SUG1": s[0] if len(s)>0 else "",
        "SUG2": s[1] if len(s)>1 else "",
        "SUG3": s[2] if len(s)>2 else "",
    })

df_diag = pd.DataFrame(rows, columns=["PARADA","OCURRENCIAS","SUG1","SUG2","SUG3"])
print(f"Total pendientes: {len(df_diag)}")
display(df_diag)


Total pendientes: 0


,PARADA,OCURRENCIAS,SUG1,SUG2,SUG3


In [9]:
# === CELDA 5: aplicar correcciones manuales a paradas ===
import re, unicodedata, numpy as np, pandas as pd

# Helpers (por si no están en memoria)
def _norm(s: str) -> str:
    s = str(s or "").strip().lower()
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c)!="Mn")
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def dms_to_decimal(dms_str):
    if not isinstance(dms_str, str):
        return (np.nan, np.nan)
    s = dms_str.strip()
    pat = re.compile(
        r"""^\s*
        (?P<lat_deg>\d{1,3})[°:\s]\s*(?P<lat_min>\d{1,2})['’:\s]\s*(?P<lat_sec>\d{1,2}(?:\.\d+)?)["”]?\s*(?P<lat_dir>[NSns])
        [,\s]+
        (?P<lon_deg>\d{1,3})[°:\s]\s*(?P<lon_min>\d{1,2})['’:\s]\s*(?P<lon_sec>\d{1,2}(?:\.\d+)?)["”]?\s*(?P<lon_dir>[EOWeow])
        \s*$""", re.X)
    m = pat.match(s)
    if not m:
        return (np.nan, np.nan)
    def comp(d, m, s): return float(d) + float(m)/60.0 + float(s)/3600.0
    lat = comp(m['lat_deg'], m['lat_min'], m['lat_sec'])
    lon = comp(m['lon_deg'], m['lon_min'], m['lon_sec'])
    if m['lat_dir'].upper() == 'S': lat *= -1
    if m['lon_dir'].upper() in ('W','O'): lon *= -1
    return (lat, lon)

def is_pending_parada(p):
    la, lo = p.get("lat"), p.get("lon")
    if la is not None and lo is not None:
        return False
    nm = str(p.get("name","")).strip()
    if not nm:
        return False
    kn = _norm(nm)
    if kn in gaz_idx:
        return False
    if kn in {"no aplica","noaplica","-","n/a"}:
        return False
    return True

# === TUS CORRECCIONES ===
# Para cada entrada: lista de alias conocidos, nombre canónico final y DMS (si diste coords).
MANUAL = [
    {"alias": ["Campo Piaggio"],
     "canonical": "Campo Piaggio",
     "dms": "31°59'11.9\"S 61°15'01.9\"W"},
    {"alias": ["Colonia Mascias","Colonia Macias"],
     "canonical": "Colonia Mascias",
     "dms": "30°47'49.9\"S 60°00'21.2\"W"},
    {"alias": ["Colonia Pujol"],
     "canonical": "Colonia Pujol",
     "dms": "31°29'02.5\"S 60°48'16.9\"W"},
    {"alias": ["Colonia Silva","Silva"],
     "canonical": "Colonia Silva",
     "dms": "30°26'56.1\"S 60°25'45.9\"W"},
    {"alias": ["Gallareta","La Gallareta"],
     "canonical": "La Gallareta",
     "dms": "29°34'58.6\"S 60°22'27.0\"W"},
    {"alias": ["Guillermina","Villa Guillermina"],
     "canonical": "Villa Guillermina",
     "dms": "28°14'25.1\"S 59°26'52.8\"W"},
    {"alias": ["Palmeras","Las Palmeras"],
     "canonical": "Las Palmeras",
     "dms": "30°37'52.9\"S 61°37'32.3\"W"},
    {"alias": ["Pueblo Casas","Casas"],
     "canonical": "Casas",
     "dms": "32°07'36.9\"S 61°32'24.4\"W"},
    {"alias": ["Punto Norte"],
     "canonical": "Punto Norte",
     "dms": "31°37'16.1\"S 60°45'50.5\"W"},
    {"alias": ["Rincón","San Jose del Rincon","San José del Rincón"],
     "canonical": "San José del Rincón",
     # No repetimos DMS si ya está en el gazetteer; si querés fijarlo igual, ponelo acá.
     "dms": None},
    {"alias": ["Rivadavia"],
     "canonical": "Rivadavia",
     "dms": "31°19'30.7\"S 61°03'03.0\"W"},
    {"alias": ["Saladero Cabal","Saladero Mariano Cabal","Saladero M. Cabal"],
     "canonical": "Saladero Mariano Cabal",
     "dms": "30°52'53.0\"S 60°02'14.1\"W"},
]

# Construyo mapa alias_normalizado -> (canonical, lat, lon)
manual_map = {}
for item in MANUAL:
    can = item["canonical"]
    la, lo = (np.nan, np.nan)
    if item.get("dms"):
        la, lo = dms_to_decimal(item["dms"])
    else:
        # intento tomar del índice si no diste DMS explícito
        k = _norm(can)
        if k in gaz_idx:
            la, lo = gaz_idx[k]
    for al in item["alias"]:
        manual_map[_norm(al)] = (can, la if not np.isnan(la) else None, lo if not np.isnan(lo) else None)

# Aplico sobre __PARADAS_LIST
aplicadas = 0
for i, L in df_full["__PARADAS_LIST"].items():
    if not isinstance(L, list):
        continue
    for p in L:
        nm = str(p.get("name","")).strip()
        if not nm:
            continue
        kn = _norm(nm)
        if kn in manual_map:
            can, la, lo = manual_map[kn]
            # renombrar
            p["name"] = can
            # setear coords (si tenemos); si no, intento gazetteer
            if (la is None or lo is None):
                kk = _norm(can)
                if kk in gaz_idx:
                    la, lo = gaz_idx[kk]
            if (la is not None) and (lo is not None):
                try:
                    p["lat"] = float(la)
                    p["lon"] = float(lo)
                except:
                    pass
            aplicadas += 1

# Actualizo también el índice con los alias (por si después hacemos lookups por nombre viejo)
for kn,(can,la,lo) in manual_map.items():
    if (la is not None) and (lo is not None):
        gaz_idx[kn] = (float(la), float(lo))

print(f"Correcciones manuales aplicadas (renombres y/o coords): {aplicadas}")

# Recontar pendientes
pend_counter = {}
for L in df_full["__PARADAS_LIST"]:
    if isinstance(L, list):
        for p in L:
            if is_pending_parada(p):
                nm = str(p.get("name","")).strip()
                pend_counter[nm] = pend_counter.get(nm, 0) + 1

restantes = sorted(pend_counter.items(), key=lambda x: (-x[1], x[0]))
print(f"Pendientes restantes: {len(restantes)}")
if restantes:
    print([nm for nm,_ in restantes][:20])  # muestro hasta 20 nombres
else:
    print("OK: sin pendientes.")

# Vista rápida de una fila cualquiera para validar que las paradas quedaron con lat/lon
# display(df_full[["LOCALIDAD DE ORIGEN","LOCALIDAD DE DESTINO","__PARADAS_LIST"]].head(2))


Correcciones manuales aplicadas (renombres y/o coords): 12
Pendientes restantes: 0
OK: sin pendientes.


In [10]:
# === CELDA 5bis (FIX): Belgrano -> Colonia Belgrano (San Martín) ===
import re, numpy as np

def _norm(s: str) -> str:
    import unicodedata, re
    s = str(s or "").strip().lower()
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c)!="Mn")
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def dms_to_decimal(dms_str):
    if not isinstance(dms_str, str):
        return (np.nan, np.nan)
    s = dms_str.strip()
    pat = re.compile(
        r"""^\s*
        (?P<lat_deg>\d{1,3})[°:\s]\s*(?P<lat_min>\d{1,2})['’:\s]\s*(?P<lat_sec>\d{1,2}(?:\.\d+)?)["”]?\s*(?P<lat_dir>[NSns])
        [,\s]+
        (?P<lon_deg>\d{1,3})[°:\s]\s*(?P<lon_min>\d{1,2})['’:\s]\s*(?P<lon_sec>\d{1,2}(?:\.\d+)?)["”]?\s*(?P<lon_dir>[EOWeow])
        \s*$""",
        re.X
    )
    m = pat.match(s)
    if not m:
        return (np.nan, np.nan)
    def comp(d, m, ss): return float(d) + float(m)/60.0 + float(ss)/3600.0
    lat = comp(m['lat_deg'], m['lat_min'], m['lat_sec'])
    lon = comp(m['lon_deg'], m['lon_min'], m['lon_sec'])
    if m['lat_dir'].upper() == 'S': lat *= -1
    if m['lon_dir'].upper() in ('W', 'O'): lon *= -1
    return (lat, lon)

def is_pending_parada(p):
    la, lo = p.get("lat"), p.get("lon")
    if la is not None and lo is not None: return False
    nm = str(p.get("name","")).strip()
    if not nm: return False
    kn = _norm(nm)
    if kn in gaz_idx: return False
    if kn in {"no aplica","noaplica","-","n/a"}: return False
    return True

# ---- dato puntual que diste
canonical = "Colonia Belgrano"
depto     = "San Martín"           # el acento no afecta porque normalizamos
dms       = "31°54'36.6\"S 61°24'08.4\"W"
lat, lon  = dms_to_decimal(dms)

# 1) actualizar índice (clave simple y con departamento) + alias "Belgrano"
gaz_idx[_norm(canonical)] = (lat, lon)
gaz_idx[_norm(canonical) + "|" + _norm(depto)] = (lat, lon)
gaz_idx["belgrano"] = (lat, lon)
gaz_idx["belgrano|"+_norm(depto)] = (lat, lon)

# 2) reescribir en __PARADAS_LIST: renombrar "Belgrano" -> "Colonia Belgrano" y poner coords
aplicadas = 0
for i, L in df_full["__PARADAS_LIST"].items():
    if not isinstance(L, list):
        continue
    for p in L:
        nm = str(p.get("name","")).strip()
        if not nm:
            continue
        if _norm(nm) in {"belgrano", _norm(canonical)}:
            p["name"] = canonical
            p["lat"]  = float(lat)
            p["lon"]  = float(lon)
            aplicadas += 1

print(f"Correcciones 'Belgrano' aplicadas: {aplicadas}")

# 3) recontar pendientes
pend_counter = {}
for L in df_full["__PARADAS_LIST"]:
    if isinstance(L, list):
        for p in L:
            if is_pending_parada(p):
                nm = str(p.get("name","")).strip()
                pend_counter[nm] = pend_counter.get(nm, 0) + 1

restantes = sorted(pend_counter.items(), key=lambda x: (-x[1], x[0]))
print(f"Pendientes restantes: {len(restantes)}")
print("OK" if not restantes else [nm for nm,_ in restantes])


Correcciones 'Belgrano' aplicadas: 2
Pendientes restantes: 0
OK


Celda 6 — Guardar correcciones en Google Sheets (minibuses)

In [11]:
# === CELDA 6: persistir correcciones en la hoja de Minibuses ===
import numpy as np, pandas as pd, re
import gspread
from gspread.utils import rowcol_to_a1

# Usa: gc, MINI_ID, MINI_TAB_PREF, df_full  (de celdas previas)

# ---------- helpers ----------
def to_paradas_geo_str(L):
    """Convierte la lista de paradas a 'lat|lon|Nombre; ...' o 'Nombre' si faltara coord."""
    if not isinstance(L, list):
        return ""
    out=[]
    for p in L:
        nm = str(p.get("name","")).strip()
        la = p.get("lat"); lo = p.get("lon")
        if la is not None and lo is not None and not (pd.isna(la) or pd.isna(lo)):
            try:
                out.append(f"{float(la):.6f}|{float(lo):.6f}|{nm}")
            except:
                out.append(nm)
        else:
            out.append(nm)
    return "; ".join(out)

def to_paradas_nombres_str(L):
    if not isinstance(L, list): return ""
    return "; ".join([str(p.get("name","")).strip() for p in L if str(p.get("name","")).strip()])

def ensure_col(ws, header_name):
    """Devuelve el índice (1-based) de la columna con ese encabezado; si no existe, la crea al final."""
    headers = ws.row_values(1)
    if header_name in headers:
        return headers.index(header_name) + 1
    # crear nueva al final
    new_col = len(headers) + 1 if headers else 1
    ws.update_cell(1, new_col, header_name)
    return new_col

def update_col(ws, col_idx, values):
    """Escribe valores (sin el header) desde la fila 2 en la columna dada."""
    n = len(values)
    if n == 0: return
    rng = f"{rowcol_to_a1(2, col_idx)}:{rowcol_to_a1(n+1, col_idx)}"
    ws.update(rng, [[v] for v in values], value_input_option="USER_ENTERED")

# ---------- abrir la hoja/pestaña ----------
sh = gc.open_by_key(MINI_ID)
tabs = [ws.title for ws in sh.worksheets()]
if 'mini_tab_used' in globals() and mini_tab_used in tabs:
    ws = sh.worksheet(mini_tab_used)
elif MINI_TAB_PREF in tabs:
    ws = sh.worksheet(MINI_TAB_PREF)
else:
    ws = sh.get_worksheet(0)

# ---------- opcional: backup de la pestaña antes de escribir ----------
MAKE_BACKUP = False  # poné True si querés duplicar antes de escribir
if MAKE_BACKUP:
    sh.duplicate_sheet(source_sheet_id=ws.id, new_sheet_name=f"{ws.title}_BACKUP")

# ---------- preparar columnas a escribir ----------
n_rows = len(df_full)
paradas_geo_col = [to_paradas_geo_str(L) for L in df_full["__PARADAS_LIST"]]
paradas_names_col = [to_paradas_nombres_str(L) for L in df_full["__PARADAS_LIST"]]

def fmt_num(x):
    try:
        v = float(x)
        if np.isnan(v): return ""
        return f"{v:.6f}"
    except:
        return ""

lat_o = [fmt_num(x) for x in df_full.get("LAT_ORIGEN", pd.Series([""]*n_rows))]
lon_o = [fmt_num(x) for x in df_full.get("LON_ORIGEN", pd.Series([""]*n_rows))]
lat_d = [fmt_num(x) for x in df_full.get("LAT_DESTINO", pd.Series([""]*n_rows))]
lon_d = [fmt_num(x) for x in df_full.get("LON_DESTINO", pd.Series([""]*n_rows))]

# ---------- asegurar/actualizar columnas ----------
col_idx_par_geo = ensure_col(ws, "PARADAS_GEO")
col_idx_par_fix = ensure_col(ws, "PARADAS INTERMEDIAS (CORREGIDAS)")
col_idx_lo_lat  = ensure_col(ws, "LAT_ORIGEN")
col_idx_lo_lon  = ensure_col(ws, "LON_ORIGEN")
col_idx_ld_lat  = ensure_col(ws, "LAT_DESTINO")
col_idx_ld_lon  = ensure_col(ws, "LON_DESTINO")

update_col(ws, col_idx_par_geo, paradas_geo_col)
update_col(ws, col_idx_par_fix, paradas_names_col)
update_col(ws, col_idx_lo_lat, lat_o)
update_col(ws, col_idx_lo_lon, lon_o)
update_col(ws, col_idx_ld_lat, lat_d)
update_col(ws, col_idx_ld_lon, lon_d)

print("✅ Actualizado en Google Sheets:")
print(f" - {ws.title} :: PARADAS_GEO")
print(f" - {ws.title} :: PARADAS INTERMEDIAS (CORREGIDAS)")
print(f" - {ws.title} :: LAT/LON de ORIGEN y DESTINO")
print(f"Filas escritas: {n_rows}")


/tmp/ipython-input-695591568.py:45: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  ws.update(rng, [[v] for v in values], value_input_option="USER_ENTERED")


✅ Actualizado en Google Sheets:
 - Salida_Paradas_Geo :: PARADAS_GEO
 - Salida_Paradas_Geo :: PARADAS INTERMEDIAS (CORREGIDAS)
 - Salida_Paradas_Geo :: LAT/LON de ORIGEN y DESTINO
Filas escritas: 26


In [12]:
# === CELDA 7 (ROBUSTA): Buscador + Mapa (Colab widgets) ===
import os, math, re, unicodedata
import numpy as np, pandas as pd
import folium
from folium.plugins import MarkerCluster
import ipywidgets as W
from IPython.display import display, clear_output, HTML

# -----------------------------
# Estética mínima para widgets
# -----------------------------
display(HTML("""
<style>
.controls-box { border:1px solid #ddd; padding:10px; border-radius:10px; margin:6px 0;}
.controls-box .widget-label { min-width: 130px !important; }
.controls-box .widget-select, .controls-box .widget-text { width: 420px !important; }
</style>
"""))

# -----------------------------
# Helpers de texto/coords
# -----------------------------
def _norm(s: str) -> str:
    s = str(s or "").strip().lower()
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c)!="Mn")
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def _clean_num(s):
    if pd.isna(s): return np.nan
    ss = str(s).strip()
    if re.match(r"^-?\d{1,3}(?:\.\d{3})+$", ss):  # -312.142.150.140.904
        ss = ss.replace(".", "")
    ss = ss.replace(",", ".")
    try: return float(ss)
    except: return np.nan

def haversine_km(a,b,c,d):
    R=6371.0088
    dphi=math.radians(c-a); dlmb=math.radians(d-b)
    p1=math.radians(a); p2=math.radians(c)
    A=math.sin(dphi/2)**2+math.cos(p1)*math.cos(p2)*math.sin(dlmb/2)**2
    return 2*R*math.asin(math.sqrt(A))

def parse_paradas_geo(texto):
    out=[]
    if not isinstance(texto, str): return out
    for it in [x.strip() for x in texto.split(";") if x.strip()]:
        parts=[p.strip() for p in it.split("|")]
        name, la, lo = None, None, None
        if len(parts)==3:
            a,b,c = parts
            try:  # lat|lon|Nombre
                la,lo = float(a.replace(",", ".")), float(b.replace(",", "."))
                name  = c
            except:
                name = a  # Nombre|lat|lon
                try:
                    la,lo = float(b.replace(",", ".")), float(c.replace(",", "."))
                except: la=lo=None
        else:
            name = it
        try:
            la = float(str(la).replace(",", ".")) if la not in (None,"","NA") else None
            lo = float(str(lo).replace(",", ".")) if lo not in (None,"","NA") else None
        except: la=lo=None
        out.append({"name": (name or "").strip(), "lat": la, "lon": lo})
    return out

# -----------------------------
# Lectura CSV publicada (fallback si no tenés open_df_sa)
# -----------------------------
def read_sheet_csv(sheet_id: str, tab_pref: str|None=None) -> tuple[str, pd.DataFrame]:
    from urllib.parse import quote
    tries=[]
    urls=[]
    if tab_pref:
        enc=quote(str(tab_pref), safe="")
        urls += [
            (f"gviz:sheet={tab_pref}",  f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={enc}"),
            (f"export:sheet={tab_pref}",f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&sheet={enc}")
        ]
    urls += [
        ("export:gid=0", f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&id={sheet_id}&gid=0"),
        ("gviz:gid=0",   f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&gid=0")
    ]
    last_err=None
    for label,url in urls:
        try:
            df=pd.read_csv(url)
            if len(df.columns)==1 and str(df.columns[0]).startswith("<!DOCTYPE"):
                tries.append(f"{label} (HTML/no publicado)")
                continue
            return (tab_pref or "Hoja 1"), df
        except Exception as e:
            last_err=e; tries.append(f"{label} ({type(e).__name__})")
    raise RuntimeError(f"No pude leer la hoja {sheet_id}. Intentos: {tries}. Último error: {last_err}")

# -----------------------------
# Cargar/asegurar datos en memoria
# -----------------------------
# 1) Gazetteer
try:
    gaz_df2  # ya viene de celdas previas
except NameError:
    gaz_df2 = pd.DataFrame()
    # Si tenés open_df_sa y GAZ_ID, lo uso; si no, pruebo CSV publicado
    if 'open_df_sa' in globals() and 'GAZ_ID' in globals():
        try:
            _, gaz_df2 = open_df_sa(GAZ_ID, globals().get('GAZ_TAB_NAME', globals().get('GAZ_TAB_PREF', None)))
        except Exception:
            pass
    if gaz_df2.empty and 'GAZ_ID' in globals():
        try:
            _, gaz_df2 = read_sheet_csv(GAZ_ID, globals().get('GAZ_TAB_NAME', globals().get('GAZ_TAB_PREF', None)))
        except Exception:
            pass

# limpiar coords del gazetteer si existen
if not gaz_df2.empty and {"centroide_lat","centroide_lon"}.issubset(set(gaz_df2.columns)):
    gaz_df2 = gaz_df2.copy()
    gaz_df2["LAT"] = gaz_df2["centroide_lat"].apply(_clean_num)
    gaz_df2["LON"] = gaz_df2["centroide_lon"].apply(_clean_num)
    gaz_df2 = gaz_df2.loc[~(gaz_df2["LAT"].isna() | gaz_df2["LON"].isna())].reset_index(drop=True)

# 2) Índice gazetteer
try:
    gaz_idx  # si ya viene armado, lo usamos
except NameError:
    gaz_idx = {}

if not gaz_idx and not gaz_df2.empty:
    idx={}
    def _add(k,la,lo):
        if k and k not in idx:
            idx[k]=(float(la),float(lo))
    name_cols=[c for c in ["nombre","localidad_censal_nombre","municipio_nombre"] if c in gaz_df2.columns]
    if name_cols:
        for _,r in gaz_df2.iterrows():
            la,lo=r["LAT"], r["LON"]
            base=set()
            for c in name_cols:
                v=r.get(c)
                if isinstance(v,str) and v.strip():
                    base.add(v.strip())
            for nm in base:
                _add(_norm(nm), la, lo)
            if "departamento_nombre" in gaz_df2.columns:
                d=r.get("departamento_nombre")
                if isinstance(d,str) and d.strip():
                    for nm in base:
                        _add(_norm(nm)+"|"+_norm(d), la, lo)
    gaz_idx = idx

def lookup_gaz(name):
    k=_norm(name)
    xy=gaz_idx.get(k)
    if xy: return xy
    for key,xy in gaz_idx.items():
        if k and k in key and "|" not in key:
            return xy
    return (np.nan, np.nan)

# 3) Minibuses
try:
    mini_df2  # ya viene de celdas previas
except NameError:
    mini_df2 = pd.DataFrame()
    if 'open_df_sa' in globals() and 'MINI_ID' in globals():
        try:
            _, mini_df2 = open_df_sa(MINI_ID, globals().get('MINI_TAB_NAME', globals().get('MINI_TAB_PREF', None)))
        except Exception:
            pass
    if mini_df2.empty and 'MINI_ID' in globals():
        try:
            _, mini_df2 = read_sheet_csv(MINI_ID, globals().get('MINI_TAB_NAME', globals().get('MINI_TAB_PREF', None)))
        except Exception:
            pass

df = mini_df2.copy()

# Asegurar numéricos en ORIGEN/DESTINO
for c in ["LAT_ORIGEN","LON_ORIGEN","LAT_DESTINO","LON_DESTINO"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# __PARADAS_LIST desde PARADAS_GEO o texto llano
if "__PARADAS_LIST" not in df.columns:
    if "PARADAS_GEO" in df.columns:
        df["__PARADAS_LIST"] = df["PARADAS_GEO"].apply(parse_paradas_geo)
    else:
        src = "PARADAS INTERMEDIAS (CORREGIDAS)" if "PARADAS INTERMEDIAS (CORREGIDAS)" in df.columns else "PARADAS INTERMEDIAS"
        def _parse_plain(s):
            out=[]
            if not isinstance(s,str): return out
            for p in re.split(r"[;|,/\\\n]+", s):
                p2=p.strip()
                if p2: out.append({"name": p2, "lat": None, "lon": None})
            return out
        df["__PARADAS_LIST"] = df.get(src, pd.Series([""]*len(df))).apply(_parse_plain)

# -----------------------------
# Listas para los dropdowns
# -----------------------------
names=set()
if not gaz_df2.empty:
    for c in ["nombre","localidad_censal_nombre","municipio_nombre"]:
        if c in gaz_df2.columns:
            names.update([str(x).strip() for x in gaz_df2[c].dropna() if str(x).strip()])
if gaz_idx:
    names.update([k.split("|")[0] for k in gaz_idx.keys() if k])
LOCALIDADES = sorted(names)

DESTINOS = ["(Todos)"] + sorted(set(str(x).strip() for x in df.get("LOCALIDAD DE DESTINO", pd.Series([])).dropna()))

# -----------------------------
# Funciones principales
# -----------------------------
def puntos_recorrido(row, df_cols=None):
    import pandas as pd
    if df_cols is None:
        try: df_cols=set(row.index)
        except Exception: df_cols=set()

    pts=[]

    # ORIGEN (nombre real; si falta coord, usar gazetteer)
    nm_origen = str(row.get("LOCALIDAD DE ORIGEN","ORIGEN"))
    la = row.get("LAT_ORIGEN", np.nan); lo = row.get("LON_ORIGEN", np.nan)
    try:
        la = float(la) if pd.notna(la) else np.nan
        lo = float(lo) if pd.notna(lo) else np.nan
    except: la=lo=np.nan
    if np.isnan(la) or np.isnan(lo):
        la, lo = lookup_gaz(nm_origen)
    if not (np.isnan(la) or np.isnan(lo)):
        pts.append(("ORIGEN", nm_origen, (la, lo)))

    # PARADAS (coords explícitas o gazetteer)
    L = row.get("__PARADAS_LIST", None)
    if L is None and "PARADAS_GEO" in df_cols:
        try: L = parse_paradas_geo(row.get("PARADAS_GEO",""))
        except: L = []
    if isinstance(L, list):
        for p in L:
            nm=str(p.get("name","")).strip()
            la,lo=p.get("lat"),p.get("lon")
            if la is None or lo is None:
                la,lo = lookup_gaz(nm)
            try:
                la,lo=float(la),float(lo)
                if not (np.isnan(la) or np.isnan(lo)):
                    pts.append(("PARADA", nm, (la,lo)))
            except: pass

    # DESTINO (nombre real; fallback gazetteer)
    nm_dest = str(row.get("LOCALIDAD DE DESTINO","DESTINO"))
    la = row.get("LAT_DESTINO", np.nan); lo = row.get("LON_DESTINO", np.nan)
    try:
        la = float(la) if pd.notna(la) else np.nan
        lo = float(lo) if pd.notna(lo) else np.nan
    except: la=lo=np.nan
    if np.isnan(la) or np.isnan(lo):
        la, lo = lookup_gaz(nm_dest)
    if not (np.isnan(la) or np.isnan(lo)):
        pts.append(("DESTINO", nm_dest, (la, lo)))

    return pts

def armar(mi_loc, dest_txt, radio_km, topn):
    # coordenadas del consultante
    lat0,lon0 = lookup_gaz(mi_loc)
    if np.isnan(lat0) or np.isnan(lon0):
        # intento exacto en gaz_df2 si existe
        if not gaz_df2.empty:
            for c in ["nombre","localidad_censal_nombre","municipio_nombre"]:
                if c in gaz_df2.columns:
                    hit = gaz_df2[gaz_df2[c].astype(str).str.strip().str.lower()==mi_loc.strip().lower()]
                    if len(hit):
                        lat0=float(hit.iloc[0]["LAT"] if "LAT" in hit.columns else hit.iloc[0]["centroide_lat"])
                        lon0=float(hit.iloc[0]["LON"] if "LON" in hit.columns else hit.iloc[0]["centroide_lon"])
                        break
    if np.isnan(lat0) or np.isnan(lon0):
        raise ValueError(f"No encontré '{mi_loc}' en el gazetteer.")

    dest_norm = _norm(dest_txt) if dest_txt and dest_txt!="(Todos)" else ""
    rows=[]

    for i,row in df.iterrows():
        if dest_norm and dest_norm not in _norm(str(row.get("LOCALIDAD DE DESTINO",""))):
            continue
        pts = puntos_recorrido(row, set(df.columns))
        if not pts:
            continue

        # --- punto de ascenso más cercano (ORIGEN o PARADA) ---
        # prioridad absoluta si mi_loc == origen (0 km)
        origen_nm = next((nm for tag,nm,_ in pts if tag=="ORIGEN"), None)
        origen_pt = next((pt for tag,nm,pt in pts if tag=="ORIGEN"), None)
        if origen_nm and _norm(origen_nm) == _norm(mi_loc):
            min_d, min_name, min_tag, min_pt = 0.0, origen_nm, "ORIGEN", origen_pt
        else:
            min_d, min_name, min_tag, min_pt = 1e9, None, None, None
            for tag, nm, (la, lo) in pts:
                if tag in {"PARADA","ORIGEN"}:
                    d = haversine_km(lat0, lon0, la, lo)
                    # tie-break: en empates (±50 m), preferir ORIGEN
                    if (d + 0.05) < min_d or (abs(d-min_d) <= 0.05 and tag=="ORIGEN"):
                        min_d, min_name, min_tag, min_pt = d, nm, tag, (la, lo)

        if min_d <= radio_km:
            # distancia de la “ruta” aproximada (ORIGEN -> PARADAS -> DESTINO)
            line=[(la,lo) for _,_,(la,lo) in pts]
            ruta_km=0.0
            for a,b in zip(line, line[1:]):
                ruta_km += haversine_km(a[0],a[1],b[0],b[1])

            rows.append({
                "idx": i+1,
                "origen": row.get("LOCALIDAD DE ORIGEN",""),
                "destino": row.get("LOCALIDAD DE DESTINO",""),
                "dias": row.get("DIAS OPERATIVOS",""),
                "punto_ascenso": f"{min_name} ({'Origen' if min_tag=='ORIGEN' else 'Parada'})",
                "dist_km": round(min_d, 2),
                "ruta_km_aprox": round(ruta_km, 1),
                "paradas_intermedias": "; ".join([nm for t,nm,_ in pts if t=="PARADA"]),
                "observaciones": row.get("OBSERVACIONES","")
            })

    # tabla
    res = (pd.DataFrame(rows).sort_values(["dist_km","ruta_km_aprox"])
           if rows else pd.DataFrame(columns=[
               "idx","origen","destino","dias","punto_ascenso","dist_km",
               "ruta_km_aprox","paradas_intermedias","observaciones"
           ]))

    # mapa
    m = folium.Map(location=[lat0,lon0], zoom_start=11, tiles="OpenStreetMap", control_scale=True)
    folium.Marker([lat0,lon0], tooltip=f"Mi localidad: {mi_loc}",
                  icon=folium.Icon(color="red", icon="home")).add_to(m)
    folium.Circle([lat0,lon0], radius=radio_km*1000, fill=False, color="red", weight=1).add_to(m)

    fg_paradas = folium.FeatureGroup(name="Puntos de ascenso").add_to(m)
    fg_rutas   = folium.FeatureGroup(name="Rutas").add_to(m)
    cluster    = MarkerCluster().add_to(fg_paradas)

    for _,r in res.head(topn).iterrows():
        row0 = df.iloc[int(r["idx"]-1)]
        pts0 = puntos_recorrido(row0, set(df.columns))
        line = [(la,lo) for _,_,(la,lo) in pts0]
        if len(line)>=2:
            folium.PolyLine(line, weight=4, color="blue", opacity=0.75,
                            tooltip=f"{r['origen']} → {r['destino']}").add_to(fg_rutas)

        asc_nm = r["punto_ascenso"].split(" (")[0]
        for tag,nm,(la,lo) in pts0:
            if nm == asc_nm and tag in {"PARADA","ORIGEN"}:
                etiqueta = "Origen (punto de ascenso)" if tag=="ORIGEN" else "Parada más cercana"
                html = (f"<b>{etiqueta}:</b> {nm}<br>"
                        f"<b>Distancia:</b> {r['dist_km']:.1f} km<br>"
                        f"<b>Origen:</b> {r['origen']}<br>"
                        f"<b>Destino:</b> {r['destino']}<br>"
                        f"<b>Días:</b> {r['dias']}")
                folium.Marker((la,lo), tooltip=f"{nm} ({r['dist_km']:.1f} km)",
                              popup=folium.Popup(html, max_width=320)).add_to(cluster)
                break

    folium.LayerControl(collapsed=False).add_to(m)
    return res, m

# -----------------------------
# Widgets y callbacks
# -----------------------------
filtro_txt   = W.Text(placeholder="Buscar localidad…", description="Buscar:")
loc_dd       = W.Dropdown(options=LOCALIDADES, description="Mi localidad:", layout=W.Layout(width="520px"))
dest_dd      = W.Dropdown(options=DESTINOS, description="Destino:", layout=W.Layout(width="420px"))
radio_sl     = W.IntSlider(value=30, min=5, max=150, step=5, description="Radio (km):", readout=True, layout=W.Layout(width="360px"))
topn_sl      = W.IntSlider(value=6,  min=1, max=15,  step=1, description="Top N:",       readout=True, layout=W.Layout(width="300px"))
buscar_btn   = W.Button(description="Buscar", button_style="primary", icon="search", layout=W.Layout(width="140px"))
reset_btn    = W.Button(description="Limpiar", icon="refresh", layout=W.Layout(width="120px"))
out          = W.Output()

def on_filter_change(change):
    q=_norm(change["new"])
    loc_dd.options = [o for o in LOCALIDADES if q in _norm(o)] if q else LOCALIDADES
filtro_txt.observe(on_filter_change, names="value")

controls = W.VBox([
    W.HBox([filtro_txt]),
    W.HBox([loc_dd]),
    W.HBox([dest_dd]),
    W.HBox([radio_sl, topn_sl, buscar_btn, reset_btn]),
], layout=W.Layout(css_classes=["controls-box"]))
display(controls, out)

@out.capture(clear_output=True, wait=True)
def on_search(b):
    mi_loc = loc_dd.value
    dest   = dest_dd.value
    radio  = int(radio_sl.value)
    topn   = int(topn_sl.value)
    res, m = armar(mi_loc, dest, radio, topn)
    # encabezado
    if len(res):
        display(HTML(f"<h4 style='margin:6px 0'>Resultados (≤ {radio} km) — mostrando {min(topn,len(res))} de {len(res)}</h4>"))
    else:
        display(HTML(f"<h4 style='margin:6px 0'>No hay recorridos con punto de ascenso a ≤ {radio} km</h4>"))
    # tabla + mapa
    display(res.head(topn))
    display(m)
    # Guardar HTML (con fallback)
    out_html = "/mnt/data/minibuses_consulta.html"
    try:
        os.makedirs(os.path.dirname(out_html), exist_ok=True)
        m.save(out_html)
    except Exception:
        out_html = "/content/minibuses_consulta.html"
        m.save(out_html)
    print("Mapa guardado en:", out_html)

def on_reset(b):
    filtro_txt.value=""
    loc_dd.options=LOCALIDADES
    radio_sl.value=30
    topn_sl.value=6
    with out: clear_output()

buscar_btn.on_click(on_search)
reset_btn.on_click(on_reset)

print("Listo. Elegí tu localidad/destino, ajustá radio y tocá ‘Buscar’. ORIGEN se considera punto de ascenso (prioridad si coincide con tu localidad).")


Output()

Listo. Elegí tu localidad/destino, ajustá radio y tocá ‘Buscar’. ORIGEN se considera punto de ascenso (prioridad si coincide con tu localidad).


In [14]:
# === CELDA 8: mapa general de TODOS los recorridos y paradas ===
import math, re, unicodedata
import numpy as np, pandas as pd
import folium
from folium.plugins import MarkerCluster
from pathlib import Path # Import Path

# --- helpers (por si no quedaron en memoria) ---
def _norm(s: str) -> str:
    s = str(s or "").strip().lower()
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c)!="Mn")
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def parse_paradas_geo(texto):
    out=[]
    if not isinstance(texto, str):
        return out
    for it in [x.strip() for x in texto.split(";") if x.strip()]:
        parts=[p.strip() for p in it.split("|")]
        name, la, lo = None, None, None
        if len(parts)==3:
            a,b,c = parts
            # intento 1: lat|lon|Nombre
            try:
                la,lo = float(a.replace(",", ".")), float(b.replace(",", "."))
                name  = c
            except:
                # intento 2: Nombre|lat|lon
                name = a
                try:
                    la,lo = float(b.replace(",", ".")), float(c.replace(",", "."))
                except:
                    la=lo=None
        else:
            name = it
        try:
            la = float(str(la).replace(",", ".")) if la not in (None,"","NA") else None
            lo = float(str(lo).replace(",", ".")) if lo not in (None,"","NA") else None
        except:
            la=lo=None
        out.append({"name": (name or "").strip(), "lat": la, "lon": lo})
    return out

def lookup_gaz(name):
    k=_norm(name)
    xy=gaz_idx.get(k) if 'gaz_idx' in globals() else None
    if xy: return xy
    # búsqueda parcial (contiene) como último recurso
    if 'gaz_idx' in globals():
        for key,xy in gaz_idx.items():
            if k and k in key and "|" not in key:
                return xy
    return (np.nan, np.nan)

def puntos_recorrido(row, df_cols):
    pts=[]
    # origen
    origen_lat, origen_lon = np.nan, np.nan
    origen_name = str(row.get("LOCALIDAD DE ORIGEN","")).strip()
    if {"LAT_ORIGEN","LON_ORIGEN"}.issubset(df_cols):
        try:
            origen_lat, origen_lon = float(row["LAT_ORIGEN"]),float(row["LON_ORIGEN"])
            if not (np.isnan(origen_lat) or np.isnan(origen_lon)):
                 # Add origin as the first point if valid
                pts.append(("ORIGEN",origen_name,(origen_lat,origen_lon)))
        except: pass

    # paradas
    L=row.get("__PARADAS_LIST", None)
    if L is None:
        if "PARADAS_GEO" in df_cols:
            L = parse_paradas_geo(row.get("PARADAS_GEO",""))
        else:
            L=[]

    # Add intermediate stops, checking for duplicates with origin
    for p in L:
        nm=str(p.get("name","")).strip()
        la,lo=p.get("lat"),p.get("lon")
        if la is None or lo is None:
            la2,lo2=lookup_gaz(nm); la,lo=la2,lo2

        is_duplicate_of_origin = False
        if not (np.isnan(origen_lat) or np.isnan(origen_lon)) and la is not None and lo is not None:
             # Check if the stop is very close to the origin
             try:
                 if haversine_km(origen_lat, origen_lon, float(la), float(lo)) < 0.1: # within 100m threshold
                     is_duplicate_of_origin = True
             except:
                 pass

        if not is_duplicate_of_origin:
            try:
                la,lo=float(la),float(lo)
                if not (np.isnan(la) or np.isnan(lo)):
                    pts.append(("PARADA",nm,(la,lo)))
            except: pass

    # destino
    destino_lat, destino_lon = np.nan, np.nan
    destino_name = str(row.get("LOCALIDAD DE DESTINO","")).strip()
    if {"LAT_DESTINO","LON_DESTINO"}.issubset(df_cols):
        try:
            destino_lat, destino_lon = float(row["LAT_DESTINO"]),float(row["LON_DESTINO"])
            if not (np.isnan(destino_lat) or np.isnan(destino_lon)):
                pts.append(("DESTINO",destino_name,(destino_lat,destino_lon)))
        except: pass

    return pts

# --- 1) Asegurar df (minibuses) en memoria; si no está, lo releemos de la hoja ---
try:
    df  # noqa
except NameError:
    # necesitamos open_df, MINI_ID, MINI_TAB_PREF de pasos previos
    gaz_title, gaz_tab_used, gaz_df2, _ = open_df(GAZ_ID,  GAZ_TAB_PREF)
    _ , _ , df, _ = open_df(MINI_ID, MINI_TAB_PREF)
    # construir __PARADAS_LIST si no estuviera
    if "__PARADAS_LIST" not in df.columns:
        if "PARADAS_GEO" in df.columns:
            df["__PARADAS_LIST"] = df["PARADAS_GEO"].apply(parse_paradas_geo)
        else:
            df["__PARADAS_LIST"] = [[] for _ in range(len(df))]

# --- 2) Recolectar todos los puntos para centrar el mapa ---
all_pts=[]
for _,row in df.iterrows():
    for tag, nm, (la,lo) in puntos_recorrido(row, set(df.columns)):
        all_pts.append((la,lo))
if not all_pts:
    # fallback al centro de Santa Fe capital
    center = (-31.633361, -60.715333)
else:
    arr = np.array(all_pts)
    center = (float(np.nanmedian(arr[:,0])), float(np.nanmedian(arr[:,1])))

# --- 3) Crear mapa y capas ---
m = folium.Map(location=list(center), zoom_start=7, tiles="OpenStreetMap", control_scale=True)

fg_origenes = folium.FeatureGroup(name="Orígenes").add_to(m)
fg_paradas  = folium.FeatureGroup(name="Paradas").add_to(m)
fg_destinos = folium.FeatureGroup(name="Destinos").add_to(m)
fg_rutas    = folium.FeatureGroup(name="Rutas").add_to(m)
cluster     = MarkerCluster(name="Cluster de paradas").add_to(fg_paradas)

# paleta simple por índice (ciclo)
palette = ["blue","green","purple","orange","darkred","cadetblue","darkblue","darkgreen","darkpurple","lightblue","lightgreen","lightgray"]

# --- 4) Dibujar todo ---
for i,row in df.iterrows():
    pts = puntos_recorrido(row, set(df.columns))
    if not pts:
        continue

    color = palette[i % len(palette)]

    # origen
    for tag,nm,(la,lo) in pts:
        if tag=="ORIGEN":
            folium.Marker((la,lo),
                          tooltip=f"Origen: {row.get('LOCALIDAD DE ORIGEN','')}",
                          icon=folium.Icon(color="green", icon="play")).add_to(fg_origenes)
            break

    # destino
    for tag,nm,(la,lo) in pts:
        if tag=="DESTINO":
            folium.Marker((la,lo),
                          tooltip=f"Destino: {row.get('LOCALIDAD DE DESTINO','')}",
                          icon=folium.Icon(color="red", icon="flag")).add_to(fg_destinos)
            break

    # paradas
    for tag,nm,(la,lo) in pts:
        if tag=="PARADA":
            info = (f"<b>Parada:</b> {nm}<br>"
                    f"<b>Origen:</b> {row.get('LOCALIDAD DE ORIGEN','')}<br>"
                    f"<b>Destino:</b> {row.get('LOCALIDAD DE DESTINO','')}<br>"
                    f"<b>Días:</b> {row.get('DIAS OPERATIVOS','')}")
            folium.Marker((la,lo), tooltip=nm, popup=folium.Popup(info, max_width=320)).add_to(cluster)

    # ruta
    line = [(la,lo) for _,_,(la,lo) in pts]
    if len(line) >= 2:
        folium.PolyLine(line, weight=3, color=color, opacity=0.7,
                        tooltip=f"{row.get('LOCALIDAD DE ORIGEN','')} → {row.get('LOCALIDAD DE DESTINO','')}").add_to(fg_rutas)

folium.LayerControl(collapsed=False).add_to(m)

# --- 5) Mostrar y guardar ---
display(m)
# Corrected path to save the map
out_html = Path("/content/minibuses_mapa_general.html")
out_html.parent.mkdir(parents=True, exist_ok=True) # Ensure directory exists
m.save(str(out_html))
print(f"✅ Mapa general guardado en: {out_html}")

✅ Mapa general guardado en: /content/minibuses_mapa_general.html


In [ ]:
# === CELDA 8bis: guardar el mapa en /content y ofrecer descarga ===
from pathlib import Path
from IPython.display import HTML

# Si venías de la Celda 8, ya tenés el objeto 'm' en memoria.
# Si reiniciaste el runtime, primero corré la Celda 8 para recrear 'm'.

out_html = Path("/content/minibuses_mapa_general.html")
out_html.parent.mkdir(parents=True, exist_ok=True)  # asegura la carpeta

m.save(str(out_html))
print(f"✅ Mapa general guardado en: {out_html}")

# Link para abrir/descargar desde Colab
HTML(f'<a href="files/{out_html.name}" target="_blank" rel="noopener">Abrir / Descargar mapa</a>')

✅ Mapa general guardado en: /content/minibuses_mapa_general.html
